In [ ]:
cd ../0_ALL_DATA/SA/

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import umap
import plotly.express as px
import plotly.graph_objects as go
import matplotlib
import matplotlib.colors as colors
import matplotlib.cbook as cbook

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.axes_grid1 import make_axes_locatable

#地図の表示
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shapereader

pd.set_option("display.max_colwidth", 500)
pd.set_option("display.max_rows", 500)
# Fontsize
plt.rcParams["font.size"] = 21
plt.rcParams['font.family'] = 'Arial'
#plt.rc('text', usetex=True)


In [ ]:
plt.rcParams["font.size"] = 21
plt.rcParams['font.family'] = 'Arial'

In [ ]:
def Spidergram_simple(data, scale, legend, color, style, label):
    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length
    
    ######## Define cycle of plotting 
    sample_nun = len(data.index)
    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)

    if(scale == "log"):
        plt.yscale("log")
    
    for num in range(sample_nun):
        if num ==0:
            plt.errorbar(t, data.values[num], color = color, linestyle=style, label = label, linewidth = 3.0)
        else:
            plt.errorbar(t, data.values[num], color = color, linestyle=style, linewidth = 3.0)
     
    if (legend == "on"):
        plt.legend(data.index)

def Spidergram_marker(data, immobile_elem, color, markeredgecolor, style, size):
    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length
    
    ######## define marker elem
    marker_row = []
    for immobile_elem_now in immobile_elem:
        num = list(columns.values).index(immobile_elem_now)
        marker_row.append(num)
    marker_row.sort() # compare with t

    ######## Define cycle of plotting 
    sample_nun = len(data.index)
    ######## Define cycle of plotting     

    ######## marker plot
    for num in range(sample_nun):
        plt.errorbar(t, data.values[num], color = color, linestyle="None", marker=style, \
                     markersize=size, markevery=marker_row, markeredgecolor=markeredgecolor, markeredgewidth=2.5)
    ######## marker plot

    

def Spidergram_error(data, model_score,scale, legend, color, style, label):
    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    sample_nun = len(data.index)
    data=data.astype(float)
    ######## Define X(columns) and X length

    ################################# To plot Error bar
    #score_elem
    model_score = model_score[columns]
    
    #error_bar_calculation log_scale
    error_plus_log = data.apply(lambda x : np.log10(x)) + model_score
    error_minus_log = data.apply(lambda x : np.log10(x)) - model_score
    raw_log = data.apply(lambda x : np.log10(x)).values
    #error_bar_calculation normal_scale
    error_plus = error_plus_log.apply(lambda x : 10**x) - 10**raw_log
    error_minus = 10**raw_log - error_minus_log.apply(lambda x : 10**x)
    error = pd.concat([error_minus, error_plus], axis = 0).values
    ################################# To plot Error bar

    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    if(scale == "log"):
        plt.yscale("log")
    
    for num in range(sample_nun):
        if num ==0:
            plt.errorbar(t, data.values[num], yerr = error, color = color, linestyle=style, label = label, capsize=5, linewidth = 3.0)
        else:
            plt.errorbar(t, data.values[num], yerr = error, color = color, linestyle=style, capsize=5, linewidth = 3.0)
     
    if (legend == "on"):
        plt.legend(data.index)

def Spidergram_fill_between(data, scale, legend, color, style, label, alpha):

    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length

    ######## Define MinMax
    # Min Maxを求めて， 上限下限の値を決める
    data_max = pd.DataFrame(data.max()).T
    data_min = pd.DataFrame(data.min()).T
    ######## Define MinMax

    ######## Define cycle of plotting 
    sample_nun = len(data.index)
    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    if(scale == "log"):
        plt.yscale("log")

    plt.fill_between(t, data_max.values[0], data_min.values[0], color = color, linestyle=style, label = label, alpha = alpha)

    if (legend == "on"):
        plt.legend(data.index)

def Spidergram_fill_immobile(columns, immobile_elem, color, alpha):
    ######## Define X(columns) and X length
    length = len(columns)
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length

    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    ######## Define X axis and label

    ######## Define fill index and fill X axis
    # 箱を準備
    elem_fill_x = []
    for elem in immobile_elem:
        # 一致しているインデックス番号を代入
        #elem_fill_x.append(list(columns).index(elem))
        num = list(columns).index(elem)
        plt.axvspan(num-0.45, num+0.45, color = color, alpha = alpha)
    ######## Define fill index

def Spidergram_ragne_as_error_bar(data, range, scale, legend, color, style, label):
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    sample_nun = len(data.index)
    data=data.astype(float)
    ######## Define X(columns) and X length

    ################################# To plot Error bar
    #score_elem
    error_plus_log = (data*range[0]).apply(lambda x : np.log10(x))
    error_minus_log = (data*range[1]).apply(lambda x : np.log10(x))
    raw_log = data.apply(lambda x : np.log10(x)).values

    #error_bar_calculation normal_scale
    error_plus = error_plus_log.apply(lambda x : 10**x) - 10**raw_log
    error_minus = 10**raw_log - error_minus_log.apply(lambda x : 10**x)
    error = pd.concat([error_minus, error_plus], axis = 0)#values
    ################################# To plot Error bar

    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    if(scale == "log"):
        plt.yscale("log")
    plt.errorbar(t, data.values.T, yerr = error.values, color = color, linestyle=style, capsize=5, linewidth = 3.0)
    
    
class SqueezedNorm(matplotlib.colors.Normalize):
    def __init__(self, vmin=None, vmax=None, mid=0, s1=2, s2=2, clip=False):
        self.vmin = vmin # minimum value
        self.mid  = mid  # middle value
        self.vmax = vmax # maximum value
        self.s1=s1; self.s2=s2
        f = lambda x, zero,vmax,s: np.abs((x-zero)/(vmax-zero))**(1./s)*0.5
        self.g = lambda x, zero,vmin,vmax, s1,s2: f(x,zero,vmax,s1)*(x>=zero) - \
                                             f(x,zero,vmin,s2)*(x<zero)+0.5
        matplotlib.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        r = self.g(value, self.mid,self.vmin,self.vmax, self.s1,self.s2)
        return np.ma.masked_array(r)


def CIA_calculation(raw_data):
    ########################################## CIA calculation
    raw_data["CaO*"] = raw_data["CaO"] - raw_data["P2O5"]/141.944/2*3/5
    raw_data["CIA*"] = 100*(raw_data["Al2O3"]/101.96)/((raw_data["Al2O3"]/101.96)+(raw_data["CaO"]/56.0774)+(raw_data["Na2O"]/61.9789)+(raw_data["K2O"]/94.2))
    raw_data["CIA"] = 100*(raw_data["Al2O3"]/101.96)/((raw_data["Al2O3"]/101.96)+(raw_data["CaO*"]/56.0774)+(raw_data["Na2O"]/61.9789)+(raw_data["K2O"]/94.2))
    ########################################## CIA calculation
    
    return raw_data

def traditional_ratio(SA_data, SA_data_RAW):

    ########################################## Ratio calculation
    SA_data["K/Th"] = SA_data_RAW["K2O"]*((10)**4)/1.204612477/ SA_data_RAW["Th"]
    SA_data["Ba/Th"] = SA_data_RAW["Ba"] / SA_data_RAW["Th"]
    SA_data["Pb/U"] = SA_data_RAW["Pb"] / SA_data_RAW["U"]
    SA_data["Th/U"] = SA_data_RAW["Th"] / SA_data_RAW["U"]
    ########################################## Ratio calculation
    return SA_data


# 0. 地図の作成

### Color setting

In [ ]:
# color setting ver230413
South_hue_order = ["U1368", "U1365", ]
South_color = ["#E4C988", "#C27664", ]
South_style=["$\circ$", "$\circ$",]

Northwest_hue_order = ["801", ]
Northwest_color = ["#84D2C5"]
Northwest_style=["$\diamond$"]

ALL_hue_order = ["U1368", "U1365", "801", ]
ALL_color = ["#E4C988", "#C27664", "#84D2C5"]
ALL_style=["$\circ$", "$\circ$", "$\diamond$"]

featured_sample_list_S = ["329-U1365E-3R-3-W 112/116", "329-U1365E-12R-1-W 19/23", "329-U1365E-7R-3-W 0/2", "329-U1365E-7R-1-W 41/45", "329-U1368F-2R-2-W 13/17", "329-U1368F-13R-2-W 108/110"]
featured_sample_color_list_S = ["#FF4500", "#FF7F50", "#FFA07A", "#FFD700", "#C99A32", "#2C1D12",]

featured_sample_list_N = ["ODP0185-0801C-028R-002/100-104", "ODP0185-0801C-040R-001/024-027", "ODP0129-0801B-041R-002/074-079", "ODP0185-0801C-046R-002/048-052"]
featured_sample_color_list_N = ["#A684D1", "#91A1ED", "#D8BFD8", "#87CEEB"]

# featured_sample_list と featured_sample_color_list を使用して試料と色をマッピングします
featured_sample_color_map = dict(zip(featured_sample_list_S+featured_sample_list_N, featured_sample_color_list_S+featured_sample_color_list_N))
featured_sample_color_map_S = dict(zip(featured_sample_list_S, featured_sample_color_list_S))
featured_sample_color_map_N = dict(zip(featured_sample_list_N, featured_sample_color_list_N))

In [ ]:
# NetCDFファイルからデータを読み取り
data = xr.open_dataset("../Seafloor Age Grid/Seton_etal_2020_PresentDay_AgeGrid.nc")
# 地図の中心を設定
center_lon = 200

# Robinsonプロジェクションを使用して地図を作成
fig = plt.figure(figsize=(24, 12))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=center_lon))

####################### 地図の基本的な描写
# 図の表示範囲をGlobalで固定
#ax.set_global()
# 背景を灰色
ax.set_facecolor('#4C444D')
# 陸地を描画
ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='black')
#ax.stock_img()
# 海岸線を描画
ax.add_feature(cfeature.COASTLINE, edgecolor='black')
####################### 地図の基本的な描写

####################### データ
# データの座標軸を取得
lons = data['x']
lats = data['y']
age_data = data['z']
# データをプロット
c = ax.pcolormesh(lons, lats, age_data, transform=ccrs.PlateCarree(), cmap='viridis_r', shading='auto', vmin=0, vmax = 200)
####################### データ

####################### Sampling point
# 指定された座標をプロット
coordinates = [
    (-27 - 55.00 / 60, -(123 + 9.65 / 60)),  # 27°55.00′S, 123°09.65′W ####U1368
    (-23 - 51.04 / 60, -(165 + 38.65 / 60)),  # 23°51.04′S,  165°38.65′W  ####U1365
    (18 + 38.54 / 60, 156 + 21.59 / 60)      # 18°38.54′N, 156°21.59′E ####801
]
for coord, now_color in zip(coordinates, ALL_color):
    lat, lon = coord
    ax.plot(lon, lat, markersize=20, marker='o', c = now_color, \
            markeredgecolor="white", markeredgewidth=4, transform=ccrs.PlateCarree())
####################### Sampling point
# カラーバーを追加
cbar = plt.colorbar(c, ax=ax, orientation='horizontal', label='Age of Oceanic Crust (Ma)', shrink=0.48, aspect=30)
cbar.ax.tick_params(labelsize=20)  # Set the tick font size

# グリッド線を追加
#ax.gridlines()
ax.set_extent([-120, 140, -90, 90], crs=ccrs.PlateCarree(central_longitude=center_lon))
plt.savefig('../Figure/1_Sample_Point_scale.jpg', bbox_inches='tight', dpi = 1000)
# 表示
plt.show()


# 1. Data Compile

### Basic setting

In [ ]:
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "P"]

Secoundary_minerals = ['vesicles (%)', 'Total Secondary mineral (%)', 'Celadonite(%)', 'Saponite (%)',
       'Fe-OX (%)', 'Carbonate (%)', 'Chalcedony (%)', 'Zeolite (%)', 'Quartz (%)']

elem_list = ['Rb', 'Ba', 'Th', 'U', 'Nb', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Zr','Ti', 'Y', 'Yb', 'Lu',]
immobile_elem=['Th', 'Nb', 'Ti', 'Zr']
major_elem = ["SiO2", "MgO", "TiO2", "CaO", "K2O", 'Na2O', 'P2O5',]
model_score = pd.read_excel('../Model_Info/Score_all.xlsx', index_col = 0, header = 0)
PM89=pd.read_excel('../List/Primitive_Mantle _ C1 Chondrite.xlsx', index_col = 0, header = 0).iloc[2]

In [ ]:
pd.read_excel('../List/Primitive_Mantle _ C1 Chondrite.xlsx', index_col = 0, header = 0)[elem_list]

## South Pacific region

### Drop
「329-U1368F-8R-1-W 31/36」が２つ変質鉱物のデータがある。変質鉱物が多い方を採用

Th empty

    329-U1365E-8R-1-W 35/39
    329-U1365E-8R-3-W 123/127

### データまとめ

    SA_data_South 
    SA_data_RAW_South
    SA_mobility_South
    SA_protolith_South

In [ ]:
########## Th mobile samples
zhang_2014_red_brown = ["329-U1365E-3R-4-W 68/70",
"329-U1365E-4R-2-W 137/142",
"329-U1365E-6R-4-W 19/23",
"329-U1365E-7R-3-W 2/4",
"329-U1365E-8R-1-W 41/47",]

zhang_2014_greenish_brown =["329-U1365E-2R-1-W 36/40",
"329-U1365E-4R-1-W 22/24",
"329-U1365E-6R-4-W 8/10",
"329-U1365E-8R-2-W 119/121",
"329-U1365E-11R-1-W 75/78",]

zhang_2014_Th_anomaly = ["329-U1365E-5R-4-W 86/90",
"329-U1365E-7R-2-W 131/135"]

# 明らかにThがずれ、Spidergram全体もおかしい試料
zhang_2014_Th_anomaly_Add_231005_BAD = [
    "329-U1365E-5R-4-W 144/149", #　可能性があるだけ、外してもOK
    "329-U1365E-6R-1-W 26/30", #　Rbだけ高い, 可能性があるだけ、外してもOK
    "329-U1365E-6R-2-W 99/103", #KのAnomalyがない
    "329-U1365E-6R-4-W 10/12", #KのAnomalyがない
    "329-U1365E-7R-1-W 41/45",
    "329-U1365E-7R-3-W 0/2", #　可能性があるだけ、外してもOK #KのAnomalyがない
    "329-U1365E-7R-4-W 0/4",
    "329-U1365E-8R-2-W 49/53",
    "329-U1365E-8R-2-W 117/119",
    "329-U1365E-8R-3-W 41/45",    
]

# 明らかにThがずれているが、Spidergam全体は正しく見える試料
zhang_2014_Th_anomaly_Add_231005 = [
    "329-U1365E-3R-2-W 52/56",
    "329-U1365E-3R-3-W 33/37",
    "329-U1365E-3R-3-W 112/116",
    "329-U1365E-6R-1-W 8/12",
    "329-U1365E-6R-2-W 19/23", # Thが完全に抜けている
    "329-U1365E-10R-1-W 46/50", #　Rbだけ高い KのAnomaly
]

################################################################################### ver 231018
zhang_2014_Th_anomaly_Add_231018 = [
# red_brown
"329-U1365E-3R-4-W 68/70",
"329-U1365E-6R-4-W 19/23",
"329-U1365E-7R-3-W 2/4",
"329-U1365E-8R-1-W 41/47",
#greenish_brown
#"329-U1365E-2R-1-W 36/40", #怪しい
"329-U1365E-6R-4-W 8/10",
# "329-U1365E-8R-2-W 119/121",#怪しい

#zhang_2014_Th_anomaly self check
"329-U1365E-5R-4-W 86/90",
"329-U1365E-7R-2-W 131/135",
    
#zhang_2014_Th_anomaly self check 231018
"329-U1365E-6R-2-W 19/23",
]
################################################################################### ver 231018


#zhang_2014_Th_anomaly_Add_231005 は採用
#zhang_2014_check_sample = zhang_2014_red_brown+zhang_2014_greenish_brown+zhang_2014_Th_anomaly
zhang_2014_check_sample = zhang_2014_Th_anomaly_Add_231018

In [ ]:
# South Pacific  path = 'South Pacific/2_PRM_Result/'
SA_data_South = pd.read_csv('South Pacific/2_PRM_Result/PM_applied_with.csv', index_col = 0, header = 0)
SA_data_RAW_South = pd.read_excel('South Pacific/1_For_PRM/230623_CompileData.xlsx', index_col = 1, header = 0)
SA_mobility_South = pd.read_csv('South Pacific/2_PRM_Result/Element_mobility_by_PRM_South_Pacific.csv', index_col = 0, header = 0)
SA_mobile_South = pd.read_csv('South Pacific/2_PRM_Result/Element_mobile_by_PRM_South_Pacific.csv', index_col = 0, header = 0)
SA_protolith_South = pd.read_csv('South Pacific/2_PRM_Result/Protolith_comp_by_PRM_(PM_normalized)_South_Pacific.csv', index_col = 0, header = 0)
SA_protolith_South_ppm = pd.read_csv('South Pacific/2_PRM_Result/Protolith_comp_by_PRM_(ppm)South_Pacific.csv', index_col = 0, header = 0).astype(float)


SA_data_South['Depth']=(SA_data_RAW_South['Top Depth (m)']+SA_data_RAW_South['Bottom Depth (m)'])/2
SA_mobility_South['Depth']=SA_data_South['Depth'].copy()
SA_mobile_South['Depth']=SA_data_South['Depth'].copy()
SA_data_RAW_South['Depth']=SA_data_South['Depth'].copy()

SA_mobility_South[['Total Secondary mineral (%)', 'Saponite (%)', 'LOI']] = SA_data_RAW_South[['Total Secondary mineral (%)', 'Saponite (%)', 'LOI']]
SA_mobile_South[['Total Secondary mineral (%)', 'Saponite (%)', 'LOI']] = SA_data_RAW_South[['Total Secondary mineral (%)', 'Saponite (%)', 'LOI']]
SA_Secoundary = SA_data_RAW_South[Secoundary_minerals]

SA_data_South['Site']="0"
SA_data_South.loc[SA_data_South.index.str.contains('U1365'), 'Site']=str('U1365')
SA_data_South.loc[SA_data_South.index.str.contains('U1368'), 'Site']=str('U1368')
SA_mobility_South['Site'] = SA_data_South['Site'].copy()

SA_data_South['area'] = 'South'
SA_data_RAW_South['area'] = 'South'
SA_mobility_South['area'] = 'South'
SA_mobile_South['area'] = 'South'
SA_data_South['REFERENCES']='Compiled by Zhang 2014'

In [ ]:
print(SA_data_South['Site'].value_counts())
print()
print(len(zhang_2014_check_sample))

## Northwest Pacific region

### データまとめ

    SA_data_Northwest
    SA_data_RAW_Northwest
    SA_mobility_Northwest
    SA_protolith_Northwest

### Northwest drop samples

In [ ]:
Northwest_Th_anomaly_Add_231020 = [
# Th anomaly in Spidergram
"ODP0185-0801C-038R-003/053-059",
"ODP0129-0801C-007R-003/043-048",
"ODP0129-0801C-005R-003/125-131", # 全体的にズレてる
"ODP0129-0801C-005R-001/095-098",
]

len(Northwest_Th_anomaly_Add_231020)

In [ ]:
# Northwest Pacific  path = 'Northwest Pacific/2_PRM_Result/'
SA_data_Northwest = pd.read_csv('Northwest Pacific/2_PRM_Result/PM_applied_with.csv', index_col = 0, header = 0)
SA_data_RAW_Northwest = pd.read_excel('Northwest Pacific/1_For_PRM/230627_CompileData.xlsx', index_col = 0, header = 0)
SA_mobility_Northwest = pd.read_csv('Northwest Pacific/2_PRM_Result/Element_mobility_by_PRM_Northwest_Pacific.csv', index_col = 0, header = 0)
SA_mobile_Northwest = pd.read_csv('Northwest Pacific/2_PRM_Result/Element_mobile_by_PRM_Northwest_Pacific.csv', index_col = 0, header = 0)
SA_protolith_Northwest = pd.read_csv('Northwest Pacific/2_PRM_Result/Protolith_comp_by_PRM_(PM_normalized)_Northwest_Pacific.csv', index_col = 0, header = 0)
SA_protolith_Northwest_ppm = pd.read_csv('Northwest Pacific/2_PRM_Result/Protolith_comp_by_PRM_(ppm)Northwest_Pacific.csv', index_col = 0, header = 0).astype(float)

SA_data_Northwest['Depth'] = SA_data_RAW_Northwest['Depth']
SA_mobility_Northwest['Depth'] = SA_data_RAW_Northwest['Depth']
SA_mobile_Northwest['Depth'] = SA_data_RAW_Northwest['Depth']
SA_protolith_Northwest['Depth'] = SA_data_RAW_Northwest['Depth']
SA_protolith_Northwest_ppm['Depth'] = SA_data_RAW_Northwest['Depth']
SA_mobility_Northwest['LOI'] = SA_data_RAW_Northwest['LOI']
SA_mobile_Northwest['LOI'] = SA_data_RAW_Northwest['LOI']


SA_data_Northwest['Site']="801"
SA_mobility_Northwest['Site']="801"

SA_data_Northwest['area'] = 'Northwest'
SA_data_RAW_Northwest['area'] = 'Northwest'
SA_mobility_Northwest['area'] = 'Northwest'
SA_mobile_Northwest['area'] = 'Northwest'
SA_data_Northwest['REFERENCES']='Compiled by Matsuno and PetDB'

### PRM without Th input

In [ ]:
#READ PRM result
SA_protolith_Zr_Ti_Nb = pd.read_csv('../SA/Compiled_data/0_PRM_Result/Zr, Ti, Nb/Protolith_comp_by_PRM_(PM_normalized)_No_database_info.csv', index_col = 0, header = 0)

### Protolith

In [ ]:
####### Protolith data
Protolith_data_PM = pd.read_excel('../Basalt comp/PM_applied_with.xlsx', index_col = 0, header = 0)
Protolith_data_RAW = pd.read_excel('../Basalt comp/Primitive_not_applied.xlsx', index_col = 0, header = 0)
index_back_arc=Protolith_data_PM[(Protolith_data_PM['SAMPLE_INFO']=="BACK-ARC_BASIN")|(Protolith_data_PM['SAMPLE_INFO']=="VOLCANIC_ARC")].index # back arc basin
Protolith_data_PM = Protolith_data_PM.drop(index_back_arc, axis=0)
Protolith_data_RAW = Protolith_data_RAW.drop(index_back_arc, axis=0)
####### Protolith data

# 2. All sample's spidergram

### Anomaly sample check

In [ ]:
elem_list_Spidergram = ['Rb', 'Ba', 'Th', 'U', 'Nb', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Zr','Ti', 'Y', 'Yb', 'Lu',]
immobile_elem=['Th', 'Nb', 'Ti', 'Zr']
#SA_data_now = SA_data[elem_list_Spidergram]
num=0

SA_data_now = SA_data_South.copy()[elem_list_Spidergram]
SA_protolith = SA_protolith_South.copy()

for index in zhang_2014_check_sample:
    SA_data_index_now =index
    num=num+1
    # Fontsize
    plt.rcParams["font.size"] = 21
    plt.rcParams['font.family'] = 'Arial'
    
    Spidergram_fill_immobile(elem_list_Spidergram, immobile_elem, '#ecc06f', 0.18)
    Spidergram_simple(pd.DataFrame(SA_data_now.loc[SA_data_index_now]).T, "log", "off","#344c5c", "--", "off")
    Spidergram_simple(pd.DataFrame(SA_protolith[elem_list_Spidergram].loc[SA_data_index_now]).T, "log", "off","#f08575", "-", "off")
    
    # Th anomaly
    Spidergram_simple(pd.DataFrame(SA_protolith_Zr_Ti_Nb[elem_list_Spidergram].loc[SA_data_index_now]).T, "log", "off","#E4C988", "-", "off")

    #Spidergram_marker(pd.DataFrame(SA_protolith[elem_list_Spidergram].loc[SA_data_index_now]).T, immobile_elem, '#f08575', '#344c5c', 'd', 16)
    ##### Define setting and savefig
    plt.ylabel('Sample / PM')
    plt.ylim(0.1, 700)
    plt.title(index)

    plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
    plt.tick_params(which = 'major', length = 7.5, width = 2)
    plt.tick_params(which = 'minor', length = 4, width = 1)

    plt.savefig("../Figure/2_1_Spidergram_Th_anomaly/"+ str(num)+'_'+ index.replace('/','') + '.pdf', bbox_inches='tight')
    plt.show()
    
SA_data_now = SA_data_Northwest.copy()[elem_list_Spidergram]
SA_protolith = SA_protolith_Northwest.copy()

for index in Northwest_Th_anomaly_Add_231020:
    SA_data_index_now =index
    num=num+1
    # Fontsize
    plt.rcParams["font.size"] = 21
    plt.rcParams['font.family'] = 'Arial'
    
    Spidergram_fill_immobile(elem_list_Spidergram, immobile_elem, '#ecc06f', 0.18)
    Spidergram_simple(pd.DataFrame(SA_data_now.loc[SA_data_index_now]).T, "log", "off","#344c5c", "--", "off")
    Spidergram_simple(pd.DataFrame(SA_protolith[elem_list_Spidergram].loc[SA_data_index_now]).T, "log", "off","#f08575", "-", "off")
    
    # Th anomaly
    Spidergram_simple(pd.DataFrame(SA_protolith_Zr_Ti_Nb[elem_list_Spidergram].loc[SA_data_index_now]).T, "log", "off","#E4C988", "-", "off")

    #Spidergram_marker(pd.DataFrame(SA_protolith[elem_list_Spidergram].loc[SA_data_index_now]).T, immobile_elem, '#f08575', '#344c5c', 'd', 16)
    ##### Define setting and savefig
    plt.ylabel('Sample / PM')
    plt.ylim(0.1, 700)
    plt.title(index)

    plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
    plt.tick_params(which = 'major', length = 7.5, width = 2)
    plt.tick_params(which = 'minor', length = 4, width = 1)

    plt.savefig("../Figure/2_1_Spidergram_Th_anomaly/"+ str(num)+'_'+ index.replace('/','') + '.pdf', bbox_inches='tight')
    plt.show()

# 3. データ整理

### South Pacific: Th anomaly data

    全てのSpidergramをチェック
    一つにProtolithを固定して確認→同じCoreであれば一致している場合もあるが、濃度が異なる場合は、PRMによる推定で個別に行った方がよい
    ThなしでPRM→Rb, Baの推定が悪くなる：PRMの精度自体も落ちるし、ThなしPRM内でもCoreの近い場所間でもバリエーションが発生

論文内でTh anomalyとされている試料については省く：zhang_2014_check_sample, Northwest_Th_anomaly_Add_231020

In [ ]:
SA_data_South.drop(zhang_2014_check_sample, inplace=True)
SA_data_RAW_South.drop(zhang_2014_check_sample, inplace=True)
SA_mobility_South.drop(zhang_2014_check_sample, inplace=True)
SA_mobile_South.drop(zhang_2014_check_sample, inplace=True)
SA_protolith_South.drop(zhang_2014_check_sample, inplace=True)
SA_protolith_South_ppm.drop(zhang_2014_check_sample, inplace=True)


SA_data_Northwest.drop(Northwest_Th_anomaly_Add_231020, inplace=True)
SA_data_RAW_Northwest.drop(Northwest_Th_anomaly_Add_231020, inplace=True)
SA_mobility_Northwest.drop(Northwest_Th_anomaly_Add_231020, inplace=True)
SA_mobile_Northwest.drop(Northwest_Th_anomaly_Add_231020, inplace=True)
SA_protolith_Northwest.drop(Northwest_Th_anomaly_Add_231020, inplace=True)
SA_protolith_Northwest_ppm.drop(Northwest_Th_anomaly_Add_231020, inplace=True)

In [ ]:
SA_data = pd.concat([SA_data_Northwest, SA_data_South])
SA_data_RAW = pd.concat([SA_data_RAW_Northwest, SA_data_RAW_South])
SA_protolith = pd.concat([SA_protolith_Northwest, SA_protolith_South])
SA_protolith_ppm = pd.concat([SA_protolith_Northwest_ppm, SA_protolith_South_ppm])
SA_mobility = pd.concat([SA_mobility_Northwest, SA_mobility_South])
SA_mobile = pd.concat([SA_mobile_Northwest, SA_mobile_South])

SA_data = CIA_calculation(SA_data)
SA_data_RAW = CIA_calculation(SA_data_RAW)

SA_data['SAMPLE_INFO']='OLD_Oceanic_Crust'
SA_data_RAW['SAMPLE_INFO']='OLD_Oceanic_Crust'
SA_protolith['SAMPLE_INFO']='OLD_Oceanic_Crust'
SA_protolith_ppm['SAMPLE_INFO']='OLD_Oceanic_Crust'
SA_mobility['SAMPLE_INFO']='OLD_Oceanic_Crust'
SA_mobile['SAMPLE_INFO']='OLD_Oceanic_Crust'

print(SA_data['Site'].value_counts())

##### ALL Spidergrams

In [ ]:
elem_list_Spidergram = ['Rb', 'Ba', 'Th', 'U', 'Nb', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Zr','Ti', 'Y', 'Yb', 'Lu',]
immobile_elem=['Th', 'Nb', 'Ti', 'Zr']
#SA_data_now = SA_data[elem_list_Spidergram]
num=0

SA_data_now = SA_data.copy()[elem_list_Spidergram]
SA_protolith = SA_protolith.copy()

for index in SA_data_now.index:
    SA_data_index_now =index
    num=num+1
    # Fontsize
    plt.rcParams["font.size"] = 21
    plt.rcParams['font.family'] = 'Arial'
    
    Spidergram_fill_immobile(elem_list_Spidergram, immobile_elem, '#ecc06f', 0.18)
    Spidergram_simple(pd.DataFrame(SA_data_now.loc[SA_data_index_now]).T, "log", "off","#344c5c", "--", "off")
    Spidergram_simple(pd.DataFrame(SA_protolith[elem_list_Spidergram].loc[SA_data_index_now]).T, "log", "off","#f08575", "-", "off")
        
    ##### Define setting and savefig
    plt.ylabel('Sample / PM')
    plt.ylim(0.1, 700)
    plt.title(index)

    plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
    plt.tick_params(which = 'major', length = 7.5, width = 2)
    plt.tick_params(which = 'minor', length = 4, width = 1)

    Location_now = SA_data['area'].loc[SA_data_index_now] #地域別にフォルダー分けする
    plt.savefig("../Figure/2_1_Spidergram_採用データ/"+Location_now+"/"+ str(num)+'_'+ index.replace('/','') + '.pdf', bbox_inches='tight')
    plt.show()

In [ ]:
SA_mobility

In [ ]:
elem_list_Spidergram = ['Rb', 'Ba', 'Th', 'U', 'Nb', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Zr','Ti', 'Y', 'Yb', 'Lu',]
immobile_elem=['Th', 'Nb', 'Ti', 'Zr']
#SA_data_now = SA_data[elem_list_Spidergram]
num=0

SA_data_now = SA_mobility.copy()[elem_list_Spidergram]

for index in SA_data_now.index:
    SA_data_index_now =index
    num=num+1
    # Fontsize
    plt.rcParams["font.size"] = 21
    plt.rcParams['font.family'] = 'Arial'
    
    Spidergram_fill_immobile(elem_list_Spidergram, immobile_elem, '#ecc06f', 0.18)
    Spidergram_simple(pd.DataFrame(SA_data_now.loc[SA_data_index_now]).T, "log", "off","#f08575", "--", "off")
    plt.axhline(y=1, color='#344c5c', linestyle='--',)
        
    ##### Define setting and savefig
    plt.ylabel('Sample / PM')
    plt.ylim(0.1, 20)
    plt.title(index)

    plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
    plt.tick_params(which = 'major', length = 7.5, width = 2)
    plt.tick_params(which = 'minor', length = 4, width = 1)

    Location_now = SA_data['area'].loc[SA_data_index_now] #地域別にフォルダー分けする
    plt.savefig("../Figure/2_2_Mobility/"+Location_now+"/"+ str(num)+'_'+ index.replace('/','') + '.pdf', bbox_inches='tight')
    plt.show()

# 4. Top of basement depth

Basement depth

    U1365 71
    U1368 13.6
    800 498.1
    801 462
    802 559.8
    1149A/B 410
    1149D 307
    197 275
    303 286
    304 335
    307 307

In [ ]:
Core_list = {
    'U1365':71,
    'U1368':9.2,
    '800':498.1,
    '801':462,
    '802':509.2,
    '1149A':410,
    '1149B':410,
    '1149C':410,
    '1149D':307,
    '197':275,
    '303':286,
    '304':335,
    '307':307,
            }

Core_name_list = {
    'U1365':'U1365',
    'U1368':'U1368',
    '800':'800',
    '801':'801',
    '802':'802',
    '1149A':'1149',
    '1149B':'1149',
    '1149C':'1149',
    '1149D':'1149',
    '197':'197',
    '303':'303',
    '304':'304',
    '307':'307',
            }
SA_data['Depth subbasement'] = 0
SA_data['Core_Site']=0
SA_data['Core_hole']=0

# 801 Lithology
Hydrothermal_801 = [[521.7, 531.2], [624, 626]]
Breccia_801 = [840, 850]
Hydrothermal_801 = [[y - 462 for y in x] for x in Hydrothermal_801]
Breccia_801 = [x - 462 for x in Breccia_801]

# Breccia U1365        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=2, alpha = 0.8)
# Breccia U1365        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=2, alpha = 0.8)

In [ ]:
Breccia_801

In [ ]:
for core_name in Core_list:
    print(core_name)
    print(SA_data.index[SA_data.index.str.contains(core_name)])
    index_now = SA_data.index[SA_data.index.str.contains(core_name)]
    basement_depth = Core_list[core_name]
    core_name_hole= Core_name_list[core_name]

    SA_data['Core_Site'].loc[index_now] = core_name
    SA_data['Core_hole'].loc[index_now] = core_name_hole
    
    SA_data_depth_now = SA_data['Depth'].loc[index_now]
    SA_data['Depth subbasement'].loc[index_now] = SA_data_depth_now - basement_depth
    
print(SA_data['Core_Site'].value_counts())

In [ ]:
#immobile_elem=['Th', 'Nb', 'Ti', 'Zr', 'TiO2']
# ppmの場合を出力； PM89と掛け算する
#SA_protolith_ppm = SA_protolith.copy() # 一度そのままコピー
#SA_protolith_ppm[elem_list] = SA_protolith[elem_list]*PM89[elem_list] # 推定した元素は掛け算でppmに戻す
#SA_protolith_ppm[immobile_elem] = SA_data_RAW[immobile_elem] # そのままppm組成を生のSAデータからコピー

# Depth subbasementを入れる処理
SA_data_RAW['Depth subbasement']=SA_data['Depth subbasement']
SA_protolith['Depth subbasementt']=SA_data['Depth subbasement']
SA_mobility['Depth subbasement']=SA_data['Depth subbasement']
SA_mobile['Depth subbasement']=SA_data['Depth subbasement']
SA_protolith_ppm['Depth subbasement']=SA_data['Depth subbasement']

# Core_Siteを入れる処理
SA_data_RAW['Core_Site']=SA_data['Core_Site']
SA_protolith['Core_Site']=SA_data['Core_Site']
SA_mobility['Core_Site']=SA_data['Core_Site']
SA_mobile['Core_Site']=SA_data['Core_Site']
SA_protolith_ppm['Core_Site']=SA_data['Core_Site']

# Core_holeを入れる処理
SA_data_RAW['Core_hole']=SA_data['Core_hole']
SA_protolith['Core_hole']=SA_data['Core_hole']
SA_mobility['Core_hole']=SA_data['Core_hole']
SA_mobile['Core_hole']=SA_data['Core_hole']
SA_protolith_ppm['Core_hole']=SA_data['Core_hole']

# K, P が計算されていないので、再計算 ver 230413
K2O_to_K = 1/141.943*30.973762*2*10000
P2O5_to_P = 1/94.2*39.0983*2*10000

# K, K2Oを入れる処理
SA_data_RAW["K"] = SA_data_RAW["K2O"]*K2O_to_K
SA_protolith["K2O"] =  SA_protolith["K"]/K2O_to_K
SA_mobile["K2O"] =  SA_mobile["K"]/K2O_to_K
SA_protolith_ppm["K2O"] =  SA_protolith_ppm["K"]/K2O_to_K

# P
SA_data_RAW["P"] = SA_data_RAW["P2O5"]*P2O5_to_P
SA_protolith["P"] = SA_protolith["P2O5"]*P2O5_to_P
SA_mobile["P"] =  SA_mobile["P2O5"]*P2O5_to_P
SA_protolith_ppm["P"] =  SA_protolith_ppm["P2O5"]*P2O5_to_P

#Ti
SA_protolith_ppm["TiO2"] =  SA_data_RAW["TiO2"]

#Ti
SA_protolith_ppm["TiO2"] =  SA_data_RAW["TiO2"]

############################### Zhang 2014 mistake refine
# "329-U1368F-13R-1-W 7/11", "Fe2O3"が負の値で登録されている→Fe2O3_totalから入力ミスであると判断
SA_data_RAW.loc["329-U1368F-13R-1-W 7/11", "Fe2O3"] = SA_data_RAW.loc["329-U1368F-13R-1-W 7/11", "Fe2O3"] * -1
############################### Zhang 2014 mistake refine

# rebuttal to Reviewer1 comment Th/U, V/Sc,
SA_data_RAW["Th/U"] = SA_data_RAW["Th"]/SA_data_RAW["U"]
SA_data_RAW["V/Sc"] = SA_data_RAW["V"]/SA_data_RAW["Sc"]

SA_protolith_ppm["TiO2"] =  SA_data_RAW["TiO2"]


data_Fe_total_mol = (SA_data_RAW["FeO"]/71.844) + (2*SA_data_RAW["Fe2O3"]/159.69)
data_Fe_3_mol = 2*SA_data_RAW["Fe2O3"]/159.69
SA_data_RAW["Fe_ratio_3"] = data_Fe_3_mol/data_Fe_total_mol

In [ ]:
# Output
SA_data.to_excel("../SA/Compiled_data/SA_data.xlsx")
SA_data_RAW.to_excel("../SA/Compiled_data/SA_data_RAW.xlsx")
SA_protolith.to_excel("../SA/Compiled_data/SA_protolith.xlsx")
SA_mobility.to_excel("../SA/Compiled_data/SA_mobility.xlsx")
SA_mobile.to_excel("../SA/Compiled_data/SA_mobile.xlsx")
SA_protolith_ppm.to_excel("../SA/Compiled_data/SA_ppm_mobile.xlsx")

# 5. Depth profile (Raw concentration)

In [ ]:
South_index = SA_mobility[SA_mobility['area']=='South'].index
West_index = SA_mobility[SA_mobility['area']=='Northwest'].index
Index_all = South_index.append(West_index)

elem_list_now = ['Rb', 'Ba',  'U', 'K2O', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Y', 'Yb', 'Lu', 'SiO2', 'MgO', 'Na2O', 'P2O5', 'CaO', 'Th', 'TiO2', 'Nb', 'Zr'] # add P and K ver230413
elem_list_protolith_ppm=['Rb', 'Ba', 'U', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Y', 'Yb', 'Lu',\
                         'SiO2', 'MgO', 'Na2O', 'P2O5', 'CaO', 'Zr', 'Th', 'TiO2', 'Nb']
# Tiを抜く

### South pacific region (SA)

In [ ]:
data=SA_data_RAW.loc[South_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_now:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        plt.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        plt.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_S))
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

        ###### Featured samples
        for sample, color in featured_sample_color_map_S.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        plt.savefig('../Figure/2_0_depth/1_South_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()
############################################## 一つ一つの図を表示

        
        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        
        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_S))
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

        ###### Featured samples
        for sample, color in featured_sample_color_map_S.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()        
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth/1_South_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      
############################################## Focusする元素のみ

############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_S))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_S.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth/1_South_Compare.pdf', bbox_inches='tight')

# 図を表示
plt.show()
############################################## Compareする元素のみ

############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_now:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_S))
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

        ###### Featured samples
        for sample, color in featured_sample_color_map_S.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth/1_South_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()    

In [ ]:
############################################## Compareする元素のみ
focus_element = ['Nb', 'La', 'Rb',]

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    
    # データの準備
    data_main = data.copy()#.drop(list(featured_sample_color_map_S))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
    """
    ###### Featured samples
    for sample, color in featured_sample_color_map_S.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    """
    ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth/1_South_Compare_without_notation.pdf', bbox_inches='tight')

# 図を表示
plt.show()
############################################## Compareする元素のみ

In [ ]:
data=SA_data_RAW.loc[South_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

############################################## rebuttal Compareする元素のみ Th/U, V/Sc
#focus_element = ['Rb', 'Nb', 'La']
focus_element = ['Th', 'U', 'V', 'Sc', 'Eu', 'Ce', 'MnO', "FeO", 'Th/U', 'V/Sc', 'Fe_ratio_3']

num_plots = len(focus_element)
num_cols = 4
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 19.5))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    ymin = data[elem_2].min()-10
    ymax = data[elem_2].max()+10

    ax = axes.flatten()[plot_index]
    ax.set_ylim(ymin, ymax)
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_S))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_S.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    elif elem_1 == "Fe_ratio_3": 
        ax.set_xlabel(r'$\frac{\mathrm{Fe}^{3+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}$ (mol)', fontsize=30)
    elif elem_1.find("/") != -1: 
        ax.set_xlabel(elem_1, fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)

    if elem_1 != "Fe_ratio_3":
        ax.set_xscale('log')


# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth/1_South_Compare_rebuttal.pdf', bbox_inches='tight')

# 図を表示
plt.show()
############################################## rebuttal Compareする元素のみ Th/U, V/Sc

In [ ]:
data=SA_data_RAW.loc[South_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

############################################## rebuttal Compareする元素のみ Th/U, V/Sc
#focus_element = ['Rb', 'Nb', 'La']
#focus_element = ['Th', 'U', 'V', 'Sc', 'Eu', 'Ce', 'MnO', "FeO", 'Th/U', 'V/Sc', "Fe_ratio_3", "S"]
#focus_element = ['Th/U', 'V/Sc', "Fe_ratio_3"]
focus_element = ['Th/U', "Fe_ratio_3"]

num_plots = len(focus_element)
num_cols = 2
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6.5))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    ymin = data[elem_2].min()-10
    ymax = data[elem_2].max()+10

    ax = axes.flatten()[plot_index]
    ax.set_ylim(ymin, ymax)
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_S))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_S.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    elif elem_1 == "Fe_ratio_3": 
        ax.set_xlabel(r'$\frac{\mathrm{Fe}^{3+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}$ (mol)', fontsize=30)
    elif elem_1.find("/") != -1: 
        ax.set_xlabel(elem_1, fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)

    if elem_1 != "Fe_ratio_3":
        ax.set_xscale('log')


# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth/1_South_Compare_rebuttal_2.pdf', bbox_inches='tight')

# 図を表示
plt.show()
############################################## rebuttal Compareする元素のみ Th/U, V/Sc

### Northwest pacific region (SA)

In [ ]:
data=SA_data_RAW.loc[West_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_now:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        plt.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        plt.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        plt.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        
        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_N))
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

        ###### Featured samples
        for sample, color in featured_sample_color_map_N.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        plt.savefig('../Figure/2_0_depth/2_West_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()

        
        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        
        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_N))
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

        ###### Featured samples
        for sample, color in featured_sample_color_map_N.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth/2_West_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      
        
############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)

    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_N))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_N.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図

    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth/2_West_Compare.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_now:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#344c5c", ec="None", alpha =0.4)
        
        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_N))
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

        ###### Featured samples
        for sample, color in featured_sample_color_map_N.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement', fontsize=25)
        ax.set_xscale('log')
        
        plot_index += 1
# 空白のサブプロットを非表示にする
if len(elem_list_now) < axes.size:
    for i in range(len(elem_list_now), axes.size):
        fig.delaxes(axes.flatten()[i])
        
# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth/2_West_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()    

In [ ]:
############################################## Compareする元素のみ
focus_element = ['Nb', 'La', 'Rb',]

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)

    # データの準備
    data_main = data.copy()#.drop(list(featured_sample_color_map_N))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
    """
    ###### Featured samples
    for sample, color in featured_sample_color_map_N.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    """
    ############################### Main 図

    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth/2_West_Compare_without_notation.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

In [ ]:
data=SA_data_RAW.loc[West_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

############################################## rebuttal Compareする元素のみ Th/U, V/Sc
#focus_element = ['Rb', 'Nb', 'La']
#focus_element = ['Th', 'U', 'V', 'Sc', 'Eu', 'Ce', 'MnO', "FeO", 'Th/U', 'V/Sc', "Fe_ratio_3", "S"]
focus_element = ['Th', 'U', 'V', 'Sc', 'Eu', 'Ce', 'MnO', "FeO", 'Th/U', 'V/Sc', "Fe_ratio_3"]


num_plots = len(focus_element)
num_cols = 4
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 19.5))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    ymin = data_all[elem_2].min()-20
    ymax = data_all[elem_2].max()+20

    ax = axes.flatten()[plot_index]
    ax.set_ylim(ymin, ymax)
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#344c5c", ec="None", alpha =0.4)

    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_N))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_N.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    elif elem_1 == "Fe_ratio_3": 
        ax.set_xlabel(r'$\frac{\mathrm{Fe}^{3+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}$ (mol)', fontsize=30)
    elif elem_1.find("/") != -1: 
        ax.set_xlabel(elem_1, fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)

    if elem_1 != "Fe_ratio_3":
        ax.set_xscale('log')
    

# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth/2_West_Compare_rebuttal.pdf', bbox_inches='tight')

# 図を表示
plt.show()
############################################## rebuttal Compareする元素のみ Th/U, V/Sc

In [ ]:
data=SA_data_RAW.loc[West_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

############################################## rebuttal Compareする元素のみ Th/U, V/Sc
#focus_element = ['Rb', 'Nb', 'La']
#focus_element = ['Th', 'U', 'V', 'Sc', 'Eu', 'Ce', 'MnO', "FeO", 'Th/U', 'V/Sc', "Fe_ratio_3", "S"]
#focus_element = ['Th/U', 'V/Sc', "Fe_ratio_3"]
focus_element = ['Th/U', "Fe_ratio_3"]

num_plots = len(focus_element)
num_cols = 2
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6.5))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    ymin = data_all[elem_2].min()-20
    ymax = data_all[elem_2].max()+20

    ax = axes.flatten()[plot_index]
    ax.set_ylim(ymin, ymax)
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#344c5c", ec="None", alpha =0.4)

    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_N))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_N.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    elif elem_1 == "Fe_ratio_3": 
        ax.set_xlabel(r'$\frac{\mathrm{Fe}^{3+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}$ (mol)', fontsize=30)
    elif elem_1.find("/") != -1: 
        ax.set_xlabel(elem_1, fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)

    if elem_1 != "Fe_ratio_3":
        ax.set_xscale('log')


    

# 不要なサブプロットを非表示にする
for i in range(num_plots, num_rows * num_cols):
    axes.flatten()[i].axis('off')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth/2_West_Compare_rebuttal_2.pdf', bbox_inches='tight')

# 図を表示
plt.show()
############################################## rebuttal Compareする元素のみ Th/U, V/Sc

## Rebuttal Fe ratio

In [ ]:
def corr_coef(elem_1, elem_2, x_, y_, x_log_flag, y_log_flag):
    # 必要な列を抽出し、欠損値を除去
    x = x_[elem_1].dropna().copy()
    y = y_[elem_2].dropna().copy()

    if x_log_flag == 1:
        x = x.apply(lambda x: np.log10(x)).dropna()
    if y_log_flag == 1:
        y = y.apply(lambda x: np.log10(x)).dropna() 
        
    # 相関係数を計算
    corr_coef = x.corr(y)

    # 結果を表示
    print(f"カテゴリー {category} の相関係数 ({elem_1} と {elem_2}): {corr_coef:.3f}")
    return corr_coef

In [ ]:
data_raw = SA_data_RAW.copy()
data_mobility = SA_mobility.copy()

# 散布図オブジェクトを保存するリスト
scatter_list = []

elem_1="Fe_ratio_3"
elem_2="Rb"

fig, ax = plt.subplots(figsize=(6, 6))
############################### Main 図
# 各カテゴリに対して散布図を作成
for category, category_c in zip(ALL_hue_order, ALL_color):
    if category=="U1368":
        zorder_setting=8
    elif category=="801":
        zorder_setting = 9
    elif category=="U1365":
        zorder_setting = 10
        
    if category!="U1368": # Old coreのみplot
        category_data_raw = data_raw[data_raw['Core_hole'] == category]        
        category_data_mobility = data_mobility[data_mobility['Core_hole'] == category]   

        # cofficient
        corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw, category_data_mobility, 0, 1)
        # ラベルに相関係数を含める
        #label = f"{category} (Corr: {corr_coef_value:.3f})"
        label = f"Corr: {corr_coef_value:.3f}"
        
        scatter = ax.scatter(category_data_raw[elem_1], category_data_mobility[elem_2], s=200, edgecolors=category_c, \
                   facecolors='none', marker="o", linewidth=3, zorder=zorder_setting, alpha=1, label=label)
        
        # 散布図オブジェクトをリストに追加
        scatter_list.append(scatter)
        
        if category=="801": # 801の深さでlabel変更 -> hydrothermal depositより下
            category_data_raw_801 = category_data_raw[category_data_raw["Depth subbasement"]<48]   
            category_data_mobility_801 = category_data_mobility[category_data_mobility["Depth subbasement"]<48] 
            
            print("< 48 m")
            corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw_801, category_data_mobility_801, 0, 1)
            #label = f"{category} (< 48 m ) (Corr: {corr_coef_value:.3f})"
            label = f"Corr: {corr_coef_value:.3f}"
            
            scatter = ax.scatter(category_data_raw_801[elem_1], category_data_mobility_801[elem_2], s=200, edgecolors="#66B4CF", \
                       facecolors='none', marker="o", linewidth=3, zorder=zorder_setting, alpha=1, label = label)
            scatter_list.append(scatter)
        
    elif category=="U1368": # U1368は
        category_data_raw = data_raw[data_raw['Core_hole'] == category]        
        category_data_mobility = data_mobility[data_mobility['Core_hole'] == category]
        
        # cofficient
        corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw, category_data_mobility, 0, 1)
        # ラベルに相関係数を含める
        #label = f"{category} (Corr: {corr_coef_value:.3f})"
        label = f"Corr: {corr_coef_value:.3f}"
        
        scatter = ax.scatter(category_data_raw[elem_1], category_data_mobility[elem_2], s=100, edgecolors="none", \
                   facecolors=category_c, marker="o", linewidth=3, zorder=zorder_setting, alpha=1, label=label)

        # 散布図オブジェクトをリストに追加
        scatter_list.append(scatter)
        
plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
#ax.get_legend().remove()
# 凡例を作成
#legend = ax.legend(handles=scatter_list, bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False, prop={'weight': 'bold'})
legend = ax.legend(handles=scatter_list, loc='lower right', frameon=True, fancybox=False, prop={'weight': 'bold', 'size': 12}, labelspacing=0.5)
for text in legend.get_texts():
    text.set_horizontalalignment('right')

plt.xlim(-0.05, 1.05)
#plt.ylim(-0.1, 25)
plt.yscale("log")

#plt.xlabel(r'$\frac{\mathrm{Fe}^{2+}\ \text{(mol)}}{\mathrm{Fe}^{2+}\ \text{(mol)} + \mathrm{Fe}^{3+}\ \text{(mol)}}$', fontsize=30)
plt.xlabel(r'$\frac{\mathrm{Fe}^{2+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}$ (mol)')
#plt.xlabel(r'$\frac{\mathrm{Fe}^{2+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}\, \mathrm{(mol)}$')
#plt.xlabel(r'$\boldsymbol{\frac{\mathrm{Fe}^{2+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}\, \mathrm{(mol)}}$')
plt.ylabel(elem_2 +' mobility', fontsize=30)

# 図を保存
plt.savefig('../Figure/Rebuttal_Fe_ratio.pdf', bbox_inches='tight')
plt.show()


In [ ]:
data_raw = SA_data_RAW.copy()
data_mobility = SA_mobility.copy()

focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P2O5"]
elem_1 = "Fe_ratio_3"

Fe_mimmax = [-0.1, 1.1]

num_plots = len(focus_element)
num_cols = 4
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(6*num_cols, 6*num_rows))
axes = axes.flatten()

for i, elem_2 in enumerate(focus_element):
    ax = axes[i]
    scatter_list = []

    #### model reproducibility
    ### calculate
    model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
    #score_elem
    # 最小値を計算
    min_error_value = 1 * 10 ** (-model_reproducibility_elem_2)
    # 最大値を計算
    max_error_value = 1 * 10 ** model_reproducibility_elem_2
    ### calculate
    ax.axhspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
    #### model reproducibility

    # Main plotting loop
    for category, category_c in zip(ALL_hue_order, ALL_color):
        if category == "U1368":
            zorder_setting = 2
        elif category == "801":
            zorder_setting = 3
        elif category == "U1365":
            zorder_setting = 4

        # mobility = 1の時の線を書く
        ax.hlines(y=1, xmin=Fe_mimmax[0], xmax=Fe_mimmax[1], linestyles=':', color='black', linewidth=8, zorder=1)
        
        if category != "U1368":  # Only plot for old core
            category_data_raw = data_raw[data_raw['Core_hole'] == category]
            category_data_mobility = data_mobility[data_mobility['Core_hole'] == category]

            # Compute correlation coefficient
            corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw, category_data_mobility, 0, 1)
            label = f"Corr: {corr_coef_value:.3f}"

            scatter = ax.scatter(
                category_data_raw[elem_1],
                category_data_mobility[elem_2],
                s=200,
                edgecolors=category_c,
                facecolors='none',
                marker="o",
                linewidth=3,
                zorder=zorder_setting,
                alpha=1,
                label=label
            )
            scatter_list.append(scatter)

            if category == "801":  # Specific label for depth < 48m
                category_data_raw_801 = category_data_raw[category_data_raw["Depth subbasement"] < 48]
                category_data_mobility_801 = category_data_mobility[category_data_mobility["Depth subbasement"] < 48]

                print("< 48 m")
                corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw_801, category_data_mobility_801, 0, 1)
                label = f"Corr: {corr_coef_value:.3f}"

                scatter = ax.scatter(
                    category_data_raw_801[elem_1],
                    category_data_mobility_801[elem_2],
                    s=200,
                    edgecolors="#66B4CF",
                    facecolors='none',
                    marker="o",
                    linewidth=3,
                    zorder=zorder_setting,
                    alpha=1,
                    label=label
                )
                scatter_list.append(scatter)

        elif category == "U1368":
            category_data_raw = data_raw[data_raw['Core_hole'] == category]
            category_data_mobility = data_mobility[data_mobility['Core_hole'] == category]

            # Compute correlation coefficient
            corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw, category_data_mobility, 0, 1)
            label = f"Corr: {corr_coef_value:.3f}"

            scatter = ax.scatter(
                category_data_raw[elem_1],
                category_data_mobility[elem_2],
                s=100,
                edgecolors="none",
                facecolors=category_c,
                marker="o",
                linewidth=3,
                zorder=zorder_setting,
                alpha=1,
                label=label
            )
            scatter_list.append(scatter)

    # Customize the axes
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.set_xlim(Fe_mimmax)
    ax.set_yscale("log")

    ax.set_xlabel(r'$\frac{\mathrm{Fe}^{3+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}$ (mol)', fontsize=30)
    if elem_2 !="P2O5":
        ax.set_ylabel(f'{elem_2} mobility', fontsize=30)
    else:
        ax.set_ylabel(f'P mobility', fontsize=30)
    

    # Create legend
    legend = ax.legend(
        handles=scatter_list,
        loc='lower right',
        frameon=True,
        fancybox=False,
        prop={'weight': 'bold', 'size': 12},
        labelspacing=0.5,
    )
    for text in legend.get_texts():
        text.set_horizontalalignment('right')

# Hide any unused subplots
for j in range(num_plots, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
# Save the figure
plt.savefig('../Figure/Rebuttal_Fe_ratio_all.pdf', bbox_inches='tight')
plt.show()


In [ ]:
data_raw = SA_data_RAW.copy()
data_mobility = SA_mobility.copy()

focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P2O5"]
elem_1 = "Fe_ratio_3"

Fe_mimmax = [-0.1, 1.1]

num_plots = len(focus_element)
num_cols = 4
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(6*num_cols, 6*num_rows))
axes = axes.flatten()

for i, elem_2 in enumerate(focus_element):
    ax = axes[i]
    scatter_list = []

    #### model reproducibility
    ### calculate
    model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
    #score_elem
    # 最小値を計算
    min_error_value = 1 * 10 ** (-model_reproducibility_elem_2)
    # 最大値を計算
    max_error_value = 1 * 10 ** model_reproducibility_elem_2
    ### calculate
    ax.axhspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
    #### model reproducibility

    # Main plotting loop
    for category, category_c in zip(ALL_hue_order, ALL_color):
        if category == "U1368":
            zorder_setting = 2
        elif category == "801":
            zorder_setting = 3
        elif category == "U1365":
            zorder_setting = 4

        # mobility = 1の時の線を書く
        ax.hlines(y=1, xmin=Fe_mimmax[0], xmax=Fe_mimmax[1], linestyles=':', color='black', linewidth=8, zorder=1)
        
        if category == "U1365":  # Only plot for old core
            category_data_raw = data_raw[data_raw['Core_hole'] == category]
            category_data_mobility = data_mobility[data_mobility['Core_hole'] == category]

            # Compute correlation coefficient
            corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw, category_data_mobility, 0, 1)
            label = f"R = {corr_coef_value:.3f}"

            scatter = ax.scatter(
                category_data_raw[elem_1],
                category_data_mobility[elem_2],
                s=200,
                edgecolors=category_c,
                facecolors='none',
                marker="o",
                linewidth=3,
                zorder=zorder_setting,
                alpha=1,
                label=label
            )
            scatter_list.append(scatter)

        elif category == "U1368":
            category_data_raw = data_raw[data_raw['Core_hole'] == category]
            category_data_mobility = data_mobility[data_mobility['Core_hole'] == category]

            # Compute correlation coefficient
            corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw, category_data_mobility, 0, 1)
            label = f"R = {corr_coef_value:.3f}"

            scatter = ax.scatter(
                category_data_raw[elem_1],
                category_data_mobility[elem_2],
                s=100,
                edgecolors="none",
                facecolors=category_c,
                marker="o",
                linewidth=3,
                zorder=zorder_setting,
                alpha=1,
                label=label
            )
            scatter_list.append(scatter)
        elif category == "801": 
            category_data_raw = data_raw[data_raw['Core_hole'] == category]
            category_data_mobility = data_mobility[data_mobility['Core_hole'] == category]
            
            ######################################### < 48 m
            category_data_raw_801 = category_data_raw[category_data_raw["Depth subbasement"] < 70]
            category_data_mobility_801 = category_data_mobility[category_data_mobility["Depth subbasement"] < 70]

            print("< 48 m")
            corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw_801, category_data_mobility_801, 0, 1)
            label = f"R = {corr_coef_value:.3f}"

            scatter = ax.scatter(
                category_data_raw_801[elem_1],
                category_data_mobility_801[elem_2],
                s=200,
                edgecolors="#66B4CF",
                facecolors='none',
                marker="o",
                linewidth=3,
                zorder=zorder_setting,
                alpha=1,
                label=label
            )
            scatter_list.append(scatter)
            ######################################### < 48 m
            ######################################### > 48 m
            category_data_raw_801 = category_data_raw[category_data_raw["Depth subbasement"] >= 70]
            category_data_mobility_801 = category_data_mobility[category_data_mobility["Depth subbasement"] >= 70]

            print(">= 48 m")
            corr_coef_value = corr_coef(elem_1, elem_2, category_data_raw_801, category_data_mobility_801, 0, 1)
            label = f"R = {corr_coef_value:.3f}"

            scatter = ax.scatter(
                category_data_raw_801[elem_1],
                category_data_mobility_801[elem_2],
                s=200,
                edgecolors="#84D2C5",
                facecolors='none',
                marker="o",
                linewidth=3,
                zorder=zorder_setting,
                alpha=1,
                label=label
            )
            scatter_list.append(scatter)
            ######################################### > 70 m
    # Customize the axes
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.set_xlim(Fe_mimmax)
    ax.set_yscale("log")

    ax.set_xlabel(r'$\frac{\mathrm{Fe}^{3+}}{\mathrm{Fe}^{2+} + \mathrm{Fe}^{3+}}$ (mol)', fontsize=30)
    if elem_2 !="P2O5":
        ax.set_ylabel(f'{elem_2} mobility', fontsize=30)
    else:
        ax.set_ylabel(f'P mobility', fontsize=30)
    

    # Create legend
    legend = ax.legend(
        handles=scatter_list,
        loc='lower right',
        frameon=True,
        fancybox=False,
        prop={'weight': 'bold', 'size': 12},
        labelspacing=0.5,
    )
    for text in legend.get_texts():
        text.set_horizontalalignment('right')

# Hide any unused subplots
for j in range(num_plots, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
# Save the figure
plt.savefig('../Figure/Rebuttal_Fe_ratio_all.pdf', bbox_inches='tight')
plt.show()


In [ ]:
category_data_raw_801

### South pacific region (RAW)

In [ ]:
###################### Raw composition
data=SA_data_RAW.loc[South_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

###################### Estimated protolith composition
data_P=SA_protolith_ppm.loc[South_index].copy()
data_all_P =SA_protolith_ppm.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        plt.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        plt.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    #pass
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
                else: # num=1
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        try:
            plt.savefig('../Figure/2_0_depth_RAW/1_South_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        except:
            pass
        plt.show()

        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
                else: # num=1
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()        
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_RAW/1_South_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    xmin_p=data_all_P[elem_1].min()
    xmax_p=data_all_P[elem_1].max()
    #################### Check min max
    if xmin_p<xmin:
        xmin = xmin_p
    if xmax_p>xmax:
        xmax = xmax_p
    #################### Check min max

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    
    # data_num=1:protolith→このとき^とする
    num=0
    for data_main in [data, data_P]:
        # データの準備
        data_main = data_main.copy()
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]
            if num==0: # 普通の時
                ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
            else: # num=1
                pass
                #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
        num=num+1
        ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth_RAW/1_South_Compare.pdf', bbox_inches='tight')

# 図を表示
plt.show()
    
############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
                else: # num=1
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_RAW/1_South_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()  

### Northwest pacific region (RAW)

In [ ]:
###################### Raw composition
data=SA_data_RAW.loc[West_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

###################### Estimated protolith composition
data_P=SA_protolith_ppm.loc[West_index].copy()
data_all_P =SA_protolith_ppm.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        plt.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        plt.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        plt.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    #pass
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
                else: # num=1
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        #plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        plt.savefig('../Figure/2_0_depth_RAW/2_West_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()

        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
                else: # num=1
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()        
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_RAW/2_West_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    xmin_p=data_all_P[elem_1].min()
    xmax_p=data_all_P[elem_1].max()
    #################### Check min max
    if xmin_p<xmin:
        xmin = xmin_p
    if xmax_p>xmax:
        xmax = xmax_p
    #################### Check min max

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
    
    # data_num=1:protolith→このとき^とする
    num=0
    for data_main in [data, data_P]:
        # データの準備
        data_main = data_main.copy()
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]
            if num==0: # 普通の時
                #pass
                ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
            else: # num=1
                pass
                #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
        num=num+1
        ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth_RAW/2_West_Compare.pdf', bbox_inches='tight')

# 図を表示
plt.show()
    
############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        
        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    #pass
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
                else: # num=1
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_RAW/2_West_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()   

### South pacific region (protolith)

In [ ]:
###################### Raw composition
data=SA_data_RAW.loc[South_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

###################### Estimated protolith composition
data_P=SA_protolith_ppm.loc[South_index].copy()
data_all_P =SA_protolith_ppm.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        plt.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        plt.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        plt.savefig('../Figure/2_0_depth_protolith/1_South_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()

        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()        
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_protolith/1_South_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    xmin_p=data_all_P[elem_1].min()
    xmax_p=data_all_P[elem_1].max()
    #################### Check min max
    if xmin_p<xmin:
        xmin = xmin_p
    if xmax_p>xmax:
        xmax = xmax_p
    #################### Check min max

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    
    # data_num=1:protolith→このとき^とする
    num=0
    for data_main in [data, data_P]:
        # データの準備
        data_main = data_main.copy()
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]
            if num==0: # 普通の時
                pass
                #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
            else: # num=1
                ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
        num=num+1
        ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth_protolith/1_South_Compare.pdf', bbox_inches='tight')

# 図を表示
plt.show()
    
############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_protolith/1_South_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()  

### Northwest pacific region (protolith)

In [ ]:
###################### Raw composition
data=SA_data_RAW.loc[West_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

###################### Estimated protolith composition
data_P=SA_protolith_ppm.loc[West_index].copy()
data_all_P =SA_protolith_ppm.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        plt.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        plt.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        plt.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        plt.savefig('../Figure/2_0_depth_protolith/2_West_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()

        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()        
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_protolith/2_West_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    xmin_p=data_all_P[elem_1].min()
    xmax_p=data_all_P[elem_1].max()
    #################### Check min max
    if xmin_p<xmin:
        xmin = xmin_p
    if xmax_p>xmax:
        xmax = xmax_p
    #################### Check min max

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
    
    # data_num=1:protolith→このとき^とする
    num=0
    for data_main in [data, data_P]:
        # データの準備
        data_main = data_main.copy()
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]
            if num==0: # 普通の時
                pass
                #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
            else: # num=1
                ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
        num=num+1
        ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth_protolith/2_West_Compare.pdf', bbox_inches='tight')

# 図を表示
plt.show()
    
############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        
        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    pass
                    #ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_protolith/2_West_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()   

### South pacific region (Both)

In [ ]:
###################### Raw composition
data=SA_data_RAW.loc[South_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

###################### Estimated protolith composition
data_P=SA_protolith_ppm.loc[South_index].copy()
data_all_P =SA_protolith_ppm.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        plt.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        plt.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        plt.savefig('../Figure/2_0_depth_both/1_South_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()

        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()        
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_both/1_South_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    xmin_p=data_all_P[elem_1].min()
    xmax_p=data_all_P[elem_1].max()
    #################### Check min max
    if xmin_p<xmin:
        xmin = xmin_p
    if xmax_p>xmax:
        xmax = xmax_p
    #################### Check min max

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    
    # data_num=1:protolith→このとき^とする
    num=0
    for data_main in [data, data_P]:
        # データの準備
        data_main = data_main.copy()
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]
            if num==0: # 普通の時
                ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
            else: # num=1
                ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
        num=num+1
        ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth_both/1_South_Compare.pdf', bbox_inches='tight')

# 図を表示
plt.show()
    
############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=xmin, xmax=xmax, linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_both/1_South_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()   

### Northwest pacific region (Both)

In [ ]:
###################### Raw composition
data=SA_data_RAW.loc[West_index].copy()
data_all =SA_data_RAW.loc[Index_all].copy()

###################### Estimated protolith composition
data_P=SA_protolith_ppm.loc[West_index].copy()
data_all_P =SA_protolith_ppm.loc[Index_all].copy()

############################################## 一つ一つの図を表示
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        
        xmin=data_all[elem_1].min()
        xmax=data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max
        
        fig, ax = plt.subplots(figsize=(6, 6))
        #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        plt.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        plt.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        plt.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        plt.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図

        plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
        plt.tick_params(which = 'major', length = 7.5, width = 2)
        plt.tick_params(which = 'minor', length = 4, width = 1)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            plt.xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            plt.xlabel(elem_1 + " (ppm)", fontsize=30)
        plt.ylabel('Depth subbasement (m)', fontsize=25)
        plt.xscale('log')
        plt.savefig('../Figure/2_0_depth_both/2_West_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()

        
############################################## Focusする元素のみ
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P"]

num_plots = 7
num_rows = 2
num_cols = 4

fig, axes = plt.subplots(num_rows, num_cols, figsize=(24, 12))

plot_index = 0
for elem_1 in focus_element:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        

        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()        
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_both/2_West_Focus.pdf', bbox_inches='tight')
# 図を表示
plt.show()      

############################################## Compareする元素のみ
focus_element = ['Rb', 'Nb', 'La']

num_plots = len(focus_element)
num_cols = 3
num_rows = (num_plots - 1) // num_cols + 1

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 6))

for plot_index, (elem_1, elem_2) in enumerate(zip(focus_element, ['Depth subbasement'] * num_plots)):
    if plot_index >= num_plots:
        break

    xmin = data_all[elem_1].min()
    xmax = data_all[elem_1].max()
    xmin_p=data_all_P[elem_1].min()
    xmax_p=data_all_P[elem_1].max()
    #################### Check min max
    if xmin_p<xmin:
        xmin = xmin_p
    if xmax_p>xmax:
        xmax = xmax_p
    #################### Check min max

    ax = axes.flatten()[plot_index]
    ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
    
    # data_num=1:protolith→このとき^とする
    num=0
    for data_main in [data, data_P]:
        # データの準備
        data_main = data_main.copy()
        # データの準備
        ############################### Main 図
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]
            if num==0: # 普通の時
                ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
            else: # num=1
                ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
        num=num+1
        ############################### Main 図
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    #ax.get_legend().remove()
    ax.invert_yaxis()
    if elem_1 in major_elem:
        ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
    else:
        ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=25)
    ax.set_xscale('log')

# サブプロット間のスペースを調整
plt.tight_layout()

# 図を保存
plt.savefig('../Figure/2_0_depth_both/2_West_Compare.pdf', bbox_inches='tight')

# 図を表示
plt.show()
    
############################################## ALL
#elem_list_now
num_plots = len(elem_list_now)
num_cols = 3
num_rows = num_plots // num_cols + int(num_plots % num_cols > 0)

fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 50))

plot_index = 0
for elem_1 in elem_list_protolith_ppm:
    for elem_2 in ['Depth subbasement']:
        if plot_index >= num_plots:
            break

        xmin = data_all[elem_1].min()
        xmax = data_all[elem_1].max()
        xmin_p=data_all_P[elem_1].min()
        xmax_p=data_all_P[elem_1].max()
        #################### Check min max
        if xmin_p<xmin:
            xmin = xmin_p
        if xmax_p>xmax:
            xmax = xmax_p
        #################### Check min max

        ax = axes[plot_index // num_cols, plot_index % num_cols]
        ax.hlines(y=0, xmin=xmin, xmax=xmax, linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        
        # data_num=1:protolith→このとき^とする
        num=0
        for data_main in [data, data_P]:
            # データの準備
            data_main = data_main.copy()
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]
                if num==0: # 普通の時
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=50, edgecolors=category_c, facecolors='none', marker="o",linewidth=3, alpha=1, zorder=9)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=120, edgecolors=category_c, facecolors='none', marker="o", alpha=0.5,linewidth=2, zorder=9)
                else: # num=1
                    ax.scatter(category_data[elem_1], category_data[elem_2], s=250, c=category_c, edgecolors="Black", linewidth=1, marker="^", zorder=10)
                    #ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="^", linewidth=3, zorder=10)
            num=num+1
            ############################### Main 図
        
        ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True)
        ax.tick_params(which='major', length=7.5, width=2)
        ax.tick_params(which='minor', length=4, width=1)
        #ax.get_legend().remove()
        ax.invert_yaxis()
        if elem_1 in major_elem:
            ax.set_xlabel(elem_1 + " (wt%)", fontsize=30)
        else:
            ax.set_xlabel(elem_1 + " (ppm)", fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=25)
        ax.set_xscale('log')

        plot_index += 1

# サブプロット間のスペースを調整
plt.tight_layout()
# 図を保存
plt.savefig('../Figure/2_0_depth_both/2_West_ALL.pdf', bbox_inches='tight')
# 図を表示
plt.show()   

# 6. Rawデータの図

In [ ]:
elem_1='Zr'
elem_2='Rb'
plt.figure(figsize=(5, 5))
ax = sns.kdeplot(data=Protolith_data_RAW, x=elem_1, y=elem_2, hue='TECTONIC SETTING', 
        log_scale=True, alpha=0.8, palette="Accent", zorder=1, legend = False)
sns.scatterplot(data=SA_data_RAW, x=elem_1, y=elem_2, hue='Core_hole', ax=ax,s=200, \
                           hue_order=ALL_hue_order, palette=ALL_color, ec="face", marker="$\circ$", zorder=2
                          )

#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
ax.get_legend().remove()

plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)

plt.xlabel(elem_1,  fontsize=30)
plt.ylabel(elem_2,  fontsize=30)

plt.savefig('../Figure/99_Supplementary/3_1_raw_scatter.pdf', bbox_inches='tight')
plt.show()

In [ ]:
elem_1='Nb'
elem_2='U'
plt.figure(figsize=(5, 5))
ax = sns.kdeplot(data=Protolith_data_RAW, x=elem_1, y=elem_2, hue='TECTONIC SETTING', 
        log_scale=True, alpha=0.8, palette="Accent", zorder=1, legend = False)
sns.scatterplot(data=SA_data_RAW, x=elem_1, y=elem_2, hue='Core_hole', ax=ax, s=200, \
                           hue_order=ALL_hue_order, palette=ALL_color, ec="face", marker="$\circ$", zorder=2
                          )

#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
ax.get_legend().remove()
plt.xlabel(elem_1,  fontsize=30)
plt.ylabel(elem_2,  fontsize=30)

plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)
plt.savefig('../Figure/99_Supplementary/3_2_raw_scatter.pdf', bbox_inches='tight')
plt.show()

In [ ]:
elem_1='Rb'
elem_2='K2O'
plt.figure(figsize=(5, 5))
ax = sns.kdeplot(data=Protolith_data_RAW, x=elem_1, y=elem_2, hue='TECTONIC SETTING', 
                 log_scale=True, alpha=0.8, palette="Accent", zorder=1, legend = False)
sns.scatterplot(data=SA_data_RAW, x=elem_1, y=elem_2, hue='Core_hole', ax=ax, s=200, \
                           hue_order=ALL_hue_order, palette=ALL_color, ec="face", marker="$\circ$", zorder=2
                          )

#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
ax.get_legend().remove()

plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)
plt.savefig('../Figure/99_Supplementary/3_2_sub_raw_scatter.pdf', bbox_inches='tight')
plt.show()

# 7. 伝統的手法：Huges Diagram & CIA

In [ ]:
South_index = SA_mobility[SA_mobility['area']=='South'].index
West_index = SA_mobility[SA_mobility['area']=='Northwest'].index
Index_all = South_index.append(West_index)

In [ ]:
########################################## Hughes (1972) diagram
SA_data["K2O+Na2O"] = SA_data.K2O+SA_data.Na2O
SA_data["100*K2O/(K2O+Na2O)"] = 100*SA_data.K2O/(SA_data.Na2O+SA_data.K2O)

plt.figure(figsize=(7, 6))

plt.plot([0, 15, 20, 30, 38], [2.5, 4, 5, 8.5, 13.5], color='black', zorder=2, alpha=0.4)
plt.plot([0, 20, 80, 85], [0.8, 2, 6.5, 13.5], color='black', zorder=2, alpha=0.4)
plt.scatter(x=SA_data["100*K2O/(K2O+Na2O)"].loc[West_index], y=SA_data["K2O+Na2O"].loc[West_index], c=SA_data['CIA'].loc[West_index], \
            cmap="jet", s=150, zorder=1, ec="face", marker="$\circ$")

plt.colorbar().set_label('CIA', size=30)

plt.ylim(0, 15)

plt.xlabel("100*K2O/(K2O+Na2O)", fontsize=30)
plt.ylabel("K2O+Na2O", fontsize=30)
plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)

plt.clim(SA_data['CIA'].loc[Index_all].min(), SA_data['CIA'].loc[Index_all].max())

plt.savefig('../Figure/99_Supplementary/3_3_Hughes (1972) diagram_West.pdf', bbox_inches='tight')
plt.show()
########################################## Hughes (1972) diagram

In [ ]:
########################################## Hughes (1972) diagram
SA_data["K2O+Na2O"] = SA_data.K2O+SA_data.Na2O
SA_data["100*K2O/(K2O+Na2O)"] = 100*SA_data.K2O/(SA_data.Na2O+SA_data.K2O)

plt.figure(figsize=(7, 6))

plt.plot([0, 15, 20, 30, 38], [2.5, 4, 5, 8.5, 13.5], color='black', zorder=2, alpha=0.4)
plt.plot([0, 20, 80, 85], [0.8, 2, 6.5, 13.5], color='black', zorder=2, alpha=0.4)
plt.scatter(x=SA_data["100*K2O/(K2O+Na2O)"].loc[South_index], y=SA_data["K2O+Na2O"].loc[South_index], c=SA_data['CIA'].loc[South_index],\
            cmap="jet", s=150, zorder=1, ec="face", marker="$\circ$")
plt.colorbar().set_label('CIA', size=30)

plt.ylim(0, 15)

plt.xlabel("100*K2O/(K2O+Na2O)", fontsize=30)
plt.ylabel("K2O+Na2O", fontsize=30)
plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)

plt.clim(SA_data['CIA'].loc[Index_all].min(), SA_data['CIA'].loc[Index_all].max())

plt.savefig('../Figure/99_Supplementary/3_3_Hughes (1972) diagram_South.pdf', bbox_inches='tight')
plt.show()
########################################## Hughes (1972) diagram

In [ ]:
########################################## Hughes (1972) diagram
SA_data["K2O+Na2O"] = SA_data.K2O+SA_data.Na2O
SA_data["100*K2O/(K2O+Na2O)"] = 100*SA_data.K2O/(SA_data.Na2O+SA_data.K2O)

plt.figure(figsize=(7, 6))

plt.plot([0, 15, 20, 30, 38], [2.5, 4, 5, 8.5, 13.5], color='black', zorder=2, alpha=0.4)
plt.plot([0, 20, 80, 85], [0.8, 2, 6.5, 13.5], color='black', zorder=2, alpha=0.4)
plt.scatter(x=SA_data["100*K2O/(K2O+Na2O)"], y=SA_data["K2O+Na2O"], c=SA_data['CIA'], cmap="jet", s=150, zorder=1\
           , ec="face", marker="$\circ$")
plt.colorbar().set_label('CIA', size=30)

plt.ylim(0, 15)

plt.xlabel("100*K2O/(K2O+Na2O)", fontsize=30)
plt.ylabel("K2O+Na2O",  fontsize=30)
plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)
plt.clim(SA_data['CIA'].loc[Index_all].min(), SA_data['CIA'].loc[Index_all].max())


plt.savefig('../Figure/99_Supplementary/3_3_Hughes (1972) diagram.pdf', bbox_inches='tight')
plt.show()
########################################## Hughes (1972) diagram

### CIA distribution

In [ ]:
# South
data=SA_data_RAW.loc[South_index].copy()

plt.figure(figsize=(6, 6))

#sns.histplot(data=SA_data, x="CIA", hue='area', linewidth=2, palette="Dark2", fill=False, alpha=0.8)
ax = sns.histplot(data=data, x="CIA", hue='Core_hole', linewidth=5, fill=False, element="step", alpha=1,\
            bins=25, legend=True, binrange=(10,60), hue_order=ALL_hue_order, palette=ALL_color,)

#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
ax.get_legend().remove()


ylim_now = plt.ylim()
plt.axvline(x=35, ymin=0, ymax=1, alpha=0.4, c="black")
plt.axvline(x=50, ymin=0, ymax=1, alpha=0.4, c="black")
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)

plt.xlabel("CIA", fontsize=30)
plt.ylabel("Count",  fontsize=30)

plt.savefig('../Figure/99_Supplementary/3_4_CIA_distribution_line_South.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# West

data=SA_data_RAW.loc[West_index].copy()

plt.figure(figsize=(6, 6))

#sns.histplot(data=SA_data, x="CIA", hue='area', linewidth=2, palette="Dark2", fill=False, alpha=0.8)
ax = sns.histplot(data=data, x="CIA", hue='Core_hole', linewidth=5, fill=False, element="step", alpha=1,\
            bins=25, legend=True, binrange=(10,60), hue_order=ALL_hue_order, palette=ALL_color,)

#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
ax.get_legend().remove()


ylim_now = plt.ylim()
plt.axvline(x=35, ymin=0, ymax=1, alpha=0.4, c="black")
plt.axvline(x=50, ymin=0, ymax=1, alpha=0.4, c="black")
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)

plt.xlabel("CIA", fontsize=30)
plt.ylabel("Count",  fontsize=30)

plt.savefig('../Figure/99_Supplementary/3_4_CIA_distribution_line_West.pdf', bbox_inches='tight')

In [ ]:
# ALL

data=SA_data_RAW.loc[Index_all].copy()

plt.figure(figsize=(6, 6))

#sns.histplot(data=SA_data, x="CIA", hue='area', linewidth=2, palette="Dark2", fill=False, alpha=0.8)
ax = sns.histplot(data=data, x="CIA", hue='Core_hole', linewidth=5, fill=False, element="step", alpha=1,\
            bins=25, legend=True, binrange=(10,60), hue_order=ALL_hue_order, palette=ALL_color)

#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
#ax.get_legend().remove()


ylim_now = plt.ylim()
plt.axvline(x=35, ymin=0, ymax=1, alpha=0.4, c="black")
plt.axvline(x=50, ymin=0, ymax=1, alpha=0.4, c="black")

plt.xlabel("CIA", fontsize=30)
plt.ylabel("Count",  fontsize=30)

plt.savefig('../Figure/99_Supplementary/3_4_CIA_distribution_line_All.pdf', bbox_inches='tight')

### K/Th vs Ba/Th 

In [ ]:
########################################## Ratio calculation
SA_data= traditional_ratio(SA_data, SA_data_RAW)
Protolith_data_RAW= traditional_ratio(Protolith_data_RAW, Protolith_data_RAW)
########################################## Ratio calculation


elem_1="Ba/Th"
elem_2="K/Th"

plt.figure(figsize=(5.5, 6))
ax = sns.kdeplot(data=Protolith_data_RAW, x=elem_1, y=elem_2, hue='TECTONIC SETTING', 
        kind="kde", log_scale=True, alpha=0.8, palette="Accent", zorder=1, legend = False)
sns.scatterplot(data=SA_data, x=elem_1, y=elem_2, hue='Core_hole', ax=ax, s=150, \
                           hue_order=ALL_hue_order, palette=ALL_color, ec="face", marker="$\circ$", zorder=2
                          )

#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=18)
ax.get_legend().remove()

plt.xlabel("Ba/Th",  fontsize=30)
plt.ylabel("K/Th",  fontsize=30)

plt.xlim(10, 10000)
plt.ylim(500, 1000000)

plt.xscale('log')
plt.yscale('log')
plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
plt.tick_params(which = 'major', length = 7.5, width = 2)
plt.tick_params(which = 'minor', length = 4, width = 1)

plt.savefig('../Figure/99_Supplementary/3_5_Bebout_emperical_diagram.pdf', bbox_inches='tight')
plt.show()

# 8. Element mobility

## Box plot for each

In [ ]:
elem_list_now = ['Rb', 'Ba',  'U', 'K', 'La', 'Ce', 'Pb', 'Sr', 'P', 'Nd', 'Y', 'Yb', 'Lu',]
order_elem_list_now = ['  ', 'Rb', 'Ba',  'U', 'K', 'La', 'Ce', 'Pb', 'Sr', 'P', 'Nd', 'Y', 'Yb', 'Lu', '   ', ] # 前後のfill_betweenを表示させるため

South_index = SA_mobility[SA_mobility['area']=='South'].index
West_index = SA_mobility[SA_mobility['area']=='Northwest'].index
Index_all = South_index.append(West_index)

Site801_index = SA_mobility[SA_mobility['Core_hole']=='801'].index
SiteU1365_index = SA_mobility[SA_mobility['Core_hole']=='U1365'].index
SiteU1368_index = SA_mobility[SA_mobility['Core_hole']=='U1368'].index

Index_list = [South_index, West_index, Index_all, Site801_index, SiteU1365_index, SiteU1368_index]
Index_list_name = ["South", "West", "All", "Site_801", "U1365", "U1368"]

In [ ]:
# 平均値をまとめる
Ave_elem_mobility = pd.DataFrame(columns=elem_list_now)
_025_elem_mobility = pd.DataFrame(columns=elem_list_now)
_075_elem_mobility = pd.DataFrame(columns=elem_list_now)

for Site_index, Site_Name in zip(Index_list, Index_list_name):
    print(Site_Name)
    ######################## 準備
    # Data
    data=SA_mobility.copy()
    data['P'] = data['P2O5'].copy()
    data_raw = data.copy()
    # Data melt
    data=pd.melt(data[elem_list_now].loc[Site_index].apply(lambda x: np.log10(x)))
    data_raw=pd.melt(data_raw[elem_list_now].loc[Site_index])

    # model information
    model_error = model_score.loc["Default_Test_mean"][elem_list_now]
    # x軸の設定
    x_model_error = np.arange(1, len(elem_list_now)+1, 1) 
    x_model_error = np.hstack((0, x_model_error, len(elem_list_now)+1)) # 元素の前後の部分の誤差を表示するため
    # y軸の設定
    y1_model_error = np.hstack((model_error.values[0], model_error.values, model_error.values[-1])) # 元素の前後の部分の誤差を表示するため-0.5部分, +0.5部分の数を追加
    y2_model_error = y1_model_error*-1 # 下側
    ######################## 準備

    ######################## 図準備
    # FigureとAxesの作成
    fig, ax = plt.subplots(figsize=(40, 10))
    ax.hlines(y=0, xmin=-1, xmax=len(elem_list_now)+1, linestyles=':', color='black',linewidth=5, zorder = 5)
    ######################## 図準備

    ######################## 図
    #sns.boxplot(x="variable", y="value", data=data, zorder = 6, ax = ax)
    sns.swarmplot(x="variable", y="value", data=data, color=".25", ax = ax, order = order_elem_list_now, size=3)
    #sns.violinplot(x="variable", y="value", data=data, scale="width", inner=None, ax = ax, order = order_elem_list_now, palette="husl")
    sns.violinplot(x="variable", y="value", data=data, inner=None, ax=ax, order=order_elem_list_now, density_norm="width", \
                  hue="variable", palette="husl")
    #sns.violinplot(x="variable", y="value", data=data, scale="width", fill=False, ax = ax, order = order_elem_list_now, palette="husl")
    #ax.errorbar(x=np.arange(0, len(elem_list_now), 1), y=[0] * len(elem_list_now), yerr = model_error.values, capsize=20, capthick=5, linewidth = 10.0, linestyle='', c='#344c5c', zorder = 8)

    ax.fill_between(x=x_model_error, y1= y1_model_error,  y2=y2_model_error, color = "#344c5c", alpha = 0.5, edgecolor="None")
    ######################## 図

    ######################## 平均値プロット図
    # 各元素の平均値の計算
    mean_values = data_raw.groupby('variable')['value'].mean()
    q1_value = data_raw.groupby('variable')['value'].quantile(0.15)
    q3_value = data_raw.groupby('variable')['value'].quantile(0.85)
    # Compile 平均＋第1四分位数＋第3四分位数
    Ave_elem_mobility.loc[Site_Name]=mean_values 
    _025_elem_mobility.loc[Site_Name]=q1_value
    _075_elem_mobility.loc[Site_Name]=q3_value
    
    # 図に平均値をプロット
    for i, elem in enumerate(order_elem_list_now):
        if elem.strip():  # 空白でない場合
            # Get value
            mean_value_log10 = np.log10(mean_values[elem])
            mean_value_raw = mean_values[elem]
            _025_value_raw =_025_elem_mobility[elem]
            _075_value_raw =_075_elem_mobility[elem]
            
            # Plot
            color_def = "#DBCAA0"
            ax.scatter(i, mean_value_log10, edgecolors=color_def, facecolors='none', marker="d", linewidth=3, s=800, zorder=10)  # 点をプロット
            ax.scatter(i, mean_value_log10, color=color_def, marker="o", s=50, zorder=10)  # 点をプロット
            #平均値の書き込み
            ax.text(i, -1.5, f'{mean_value_raw:.1f}', color='black', ha='center', va='center', fontsize=40)
    ######################## 平均値プロット図

    plt.xlabel('', fontsize=30)
    plt.ylabel(r'$\log_{10}$(Mobility)', fontsize=50) # LaTexの書き方
    plt.xticks(fontsize=60)
    plt.yticks(fontsize=50)
    plt.xlim(0.5, len(elem_list_now)+0.5)
    plt.ylim(-2, 3)

    # Scatter plot settings
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=False, right=True)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    
    plt.savefig('../Figure/3_Mobility_Violin/3_mobility_violine_'+Site_Name+'.pdf', bbox_inches='tight')
    plt.show()

# Ave_elem_mobility を保存
Ave_elem_mobility.to_excel("../Figure/3_Mobility_Violin/0_Ave_elem_mobility.xlsx")

## Depth profile

In [ ]:
South_index = SA_mobility[SA_mobility['area']=='South'].index
West_index = SA_mobility[SA_mobility['area']=='Northwest'].index

elem_list_now = ['Rb', 'Ba',  'U', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Y', 'Yb', 'Lu', 'SiO2', 'MgO', 'Na2O', 'P2O5', 'CaO']

In [ ]:
data=SA_mobility.loc[South_index].copy()
data_all=SA_mobility.copy()

for elem_1 in elem_list_now:
    for elem_2 in ['Depth subbasement']:
        
        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_S))
        # データの準備
        ############################### Main 図
        # 図と軸の作成
        fig, ax = plt.subplots(figsize=(6, 6))
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
        
        ###### Featured samples
        for sample, color in featured_sample_color_map_S.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図
        
        # line
        ax.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        ax.hlines(y=0, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=8)
        ax.hlines(y=23, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        ax.hlines(y=36, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
        #### model reproducibility
        ### calculate
        model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
        #score_elem
        # 最小値を計算
        min_error_value = 1 * 10 ** (-model_reproducibility_elem_1)
        # 最大値を計算
        max_error_value = 1 * 10 ** model_reproducibility_elem_1
        ### calculate
        ax.axvspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
        #### model reproducibility

        #scatter plot setting
        ax.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True, labelsize=20)
        ax.tick_params(which = 'major', length = 7.5, width = 2)
        ax.tick_params(which = 'minor', length = 4, width = 1)
        ax.invert_yaxis()
        if elem_1=="P2O5":
            ax.set_xlabel("P mobility",  fontsize=30)
        else:
            ax.set_xlabel(elem_1+" mobility",  fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=30)
        ax.set_xscale('log')
        
        # kde
        # 上部の軸を追加
        divider = make_axes_locatable(plt.gca())
        ax_kde = divider.append_axes("top", size="20%", pad=0.0) 
        ax_kde.set(xlim=ax.get_xlim())
        ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
        
        # x軸に沿ったKDEプロットを描画
        sns.kdeplot(data=data.copy(), x=elem_1, ax=ax_kde, alpha=1, linewidth=3.5, log_scale=True,\
                    hue='Core_hole', hue_order=ALL_hue_order, palette=ALL_color, legend=False, common_norm=False)
        ax_kde.set(xlim=ax.get_xlim())
        ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
        ax_kde.xaxis.set_visible(False)
        
        plt.savefig('../Figure/4_depth/1_South_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()
        
        
############################################################### ALL
# 列数の計算
num_cols = min(len(elem_list_now), 3)

# 行数と列数に基づいて図のサイズを計算
fig_width = 6 * num_cols
fig_height = 6 * ((len(elem_list_now) - 1) // num_cols + 1)

fig, axes = plt.subplots(nrows=(len(elem_list_now) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

for i, elem_1 in enumerate(elem_list_now):
    row = i // num_cols
    col = i % num_cols
    
    ax = axes[row, col]

    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_S))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_S.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
            
    # Line
    ax.vlines(x=1, ymin=data['Depth subbasement'].min(), ymax=data['Depth subbasement'].max(), linestyles=':', color='black', linewidth=3.5)
    ax.hlines(y=0, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    #### model reproducibility
    ### calculate
    model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
    #score_elem
    # 最小値を計算
    min_error_value = 1 * 10 ** (-model_reproducibility_elem_1)
    # 最大値を計算
    max_error_value = 1 * 10 ** model_reproducibility_elem_1
    ### calculate
    ax.axvspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
    #### model reproducibility
    
    # Scatter plot settings
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.invert_yaxis()
    if elem_1=="P2O5":
        ax.set_xlabel("P mobility",  fontsize=30)
    else:
        ax.set_xlabel(elem_1+" mobility",  fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=30)
    ax.set_xscale('log')
    
    # KDE
    divider = make_axes_locatable(ax)
    ax_kde = divider.append_axes("top", size="20%", pad=0.0) 
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    
    sns.kdeplot(data=data.copy(), x=elem_1, ax=ax_kde, alpha=1, linewidth=3.5, log_scale=True, hue='Core_hole', hue_order=ALL_hue_order, palette=ALL_color, legend=False, common_norm=False)
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    ax_kde.xaxis.set_visible(False)

    if elem_1 == "Sr":
        ax.set_xlim([0.6, 2.15])
        ax_kde.set_xlim([0.6, 2.15])
    else:
        pass

# 空白のサブプロットを非表示にする
if len(elem_list_now) < axes.size:
    for i in range(len(elem_list_now), axes.size):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.savefig('../Figure/4_depth/1_South_ALL.pdf', bbox_inches='tight')
plt.show()

############################################################### FOCUS
# 列数の計算
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P2O5"]
num_cols = min(len(focus_element), 4)

# 行数と列数に基づいて図のサイズを計算
fig_width = 6 * num_cols
fig_height = 6 * ((len(focus_element) - 1) // num_cols + 1)

fig, axes = plt.subplots(nrows=(len(focus_element) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

for i, elem_1 in enumerate(focus_element):
    row = i // num_cols
    col = i % num_cols
    
    ax = axes[row, col]
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_S))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_S.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
            
    # Line
    ax.vlines(x=1, ymin=data['Depth subbasement'].min(), ymax=data['Depth subbasement'].max(), linestyles=':', color='black', linewidth=3.5)
    ax.hlines(y=0, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=8)
    ax.hlines(y=23, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    ax.hlines(y=36, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color="#C27664", linewidth=5, alpha = 0.8)
    #### model reproducibility
    ### calculate
    model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
    #score_elem
    # 最小値を計算
    min_error_value = 1 * 10 ** (-model_reproducibility_elem_1)
    # 最大値を計算
    max_error_value = 1 * 10 ** model_reproducibility_elem_1
    ### calculate
    ax.axvspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
    #### model reproducibility
    
    # Scatter plot settings
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.invert_yaxis()
    if elem_1=="P2O5":
        ax.set_xlabel("P mobility",  fontsize=30)
    else:
        ax.set_xlabel(elem_1+" mobility",  fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=30)
    ax.set_xscale('log')
    
    # KDE
    divider = make_axes_locatable(ax)
    ax_kde = divider.append_axes("top", size="20%", pad=0.0) 
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    
    sns.kdeplot(data=data.copy(), x=elem_1, ax=ax_kde, alpha=1, linewidth=3.5, log_scale=True, hue='Core_hole', hue_order=ALL_hue_order, palette=ALL_color, legend=False, common_norm=False)
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    ax_kde.xaxis.set_visible(False)

    if elem_1 == "Sr":
        ax.set_xlim([0.6, 2.15])
        ax_kde.set_xlim([0.6, 2.15])
    else:
        pass
    
# 空白のサブプロットを非表示にする
if len(focus_element) < axes.size:
    for i in range(len(focus_element), axes.size):
        fig.delaxes(axes.flatten()[i])
        
plt.tight_layout()
plt.savefig('../Figure/4_depth/1_South_Focus.pdf', bbox_inches='tight')
plt.show()


In [ ]:
data=SA_mobility.loc[West_index].copy()
data_all=SA_mobility.copy()

for elem_1 in elem_list_now:
    for elem_2 in ['Depth subbasement']:
        
        # データの準備
        data_main = data.copy().drop(list(featured_sample_color_map_N))
        # データの準備
        ############################### Main 図
        # 図と軸の作成
        fig, ax = plt.subplots(figsize=(6, 6))
        # 各カテゴリに対して散布図を作成
        for category, category_c in zip(ALL_hue_order, ALL_color):
            category_data = data_main[data_main['Core_hole'] == category]            
            ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)
        
        ###### Featured samples
        for sample, color in featured_sample_color_map_N.items():
            try:
                ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                           edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
            except:
                pass
        ###### Featured samples
        ############################### Main 図
         
        # line
        ax.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
        ax.hlines(y=0, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=8)
        # Hydrothermal
        ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
        ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
        # Breccia Unit
        ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)
        
        #### model reproducibility
        ### calculate
        model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
        #score_elem
        # 最小値を計算
        min_error_value = 1 * 10 ** (-model_reproducibility_elem_1)
        # 最大値を計算
        max_error_value = 1 * 10 ** model_reproducibility_elem_1
        ### calculate
        ax.axvspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
        #### model reproducibility
        
        #scatter plot setting
        ax.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True, labelsize=20)
        ax.tick_params(which = 'major', length = 7.5, width = 2)
        ax.tick_params(which = 'minor', length = 4, width = 1)
        ax.invert_yaxis()
        if elem_1=="P2O5":
            ax.set_xlabel("P mobility",  fontsize=30)
        else:
            ax.set_xlabel(elem_1+" mobility",  fontsize=30)
        ax.set_ylabel('Depth subbasement (m)', fontsize=30)
        ax.set_xscale('log')
        
        # kde
        # 上部の軸を追加
        divider = make_axes_locatable(plt.gca())
        ax_kde = divider.append_axes("top", size="20%", pad=0.0) 
        ax_kde.set(xlim=ax.get_xlim())
        ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
        
        # x軸に沿ったKDEプロットを描画
        sns.kdeplot(data=data.copy(), x=elem_1, ax=ax_kde, alpha=1, linewidth=3.5, log_scale=True,\
                    hue='Core_hole', hue_order=ALL_hue_order, palette=ALL_color, legend=False, common_norm=False)
        ax_kde.set(xlim=ax.get_xlim())
        ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
        ax_kde.xaxis.set_visible(False)
        
        plt.tight_layout()
        plt.savefig('../Figure/4_depth/2_West_'+elem_2+ '_' +elem_1+ '.pdf', bbox_inches='tight')
        plt.show()
        
        
############################################################### ALL
# 列数の計算
num_cols = min(len(elem_list_now), 3)

# 行数と列数に基づいて図のサイズを計算
fig_width = 6 * num_cols
fig_height = 6 * ((len(elem_list_now) - 1) // num_cols + 1)

fig, axes = plt.subplots(nrows=(len(elem_list_now) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

for i, elem_1 in enumerate(elem_list_now):
    row = i // num_cols
    col = i % num_cols
    
    ax = axes[row, col]
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_N))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_N.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図

    # Line
    ax.vlines(x=1, ymin=data['Depth subbasement'].min(), ymax=data['Depth subbasement'].max(), linestyles=':', color='black', linewidth=3.5)
    ax.hlines(y=0, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)

    #### model reproducibility
    ### calculate
    model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
    #score_elem
    # 最小値を計算
    min_error_value = 1 * 10 ** (-model_reproducibility_elem_1)
    # 最大値を計算
    max_error_value = 1 * 10 ** model_reproducibility_elem_1
    ### calculate
    ax.axvspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
    #### model reproducibility
    
    # Scatter plot settings
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.invert_yaxis()
    if elem_1=="P2O5":
        ax.set_xlabel("P mobility",  fontsize=30)
    else:
        ax.set_xlabel(elem_1+" mobility",  fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=30)
    ax.set_xscale('log')
    
    # KDE
    divider = make_axes_locatable(ax)
    ax_kde = divider.append_axes("top", size="20%", pad=0.0) 
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    
    sns.kdeplot(data=data.copy(), x=elem_1, ax=ax_kde, alpha=1, linewidth=3.5, log_scale=True, hue='Core_hole', hue_order=ALL_hue_order, palette=ALL_color, legend=False, common_norm=False)
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    ax_kde.xaxis.set_visible(False)

# 空白のサブプロットを非表示にする
if len(elem_list_now) < axes.size:
    for i in range(len(elem_list_now), axes.size):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.savefig('../Figure/4_depth/2_West_ALL.pdf', bbox_inches='tight')
plt.show()

############################################################### FOCUS
# 列数の計算
focus_element = ['Rb', 'Ba', 'U', 'K', 'Pb', "Sr", "P2O5"]
num_cols = min(len(focus_element), 4)

# 行数と列数に基づいて図のサイズを計算
fig_width = 6 * num_cols
fig_height = 6 * ((len(focus_element) - 1) // num_cols + 1)

fig, axes = plt.subplots(nrows=(len(focus_element) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

for i, elem_1 in enumerate(focus_element):
    row = i // num_cols
    col = i % num_cols
    
    ax = axes[row, col]
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_N))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data['Depth subbasement'], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_N.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図
    # Line
    ax.vlines(x=1, ymin=data['Depth subbasement'].min(), ymax=data['Depth subbasement'].max(), linestyles=':', color='black', linewidth=3.5)
    ax.hlines(y=0, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=8)
    # Hydrothermal
    ax.axhspan(Hydrothermal_801[0][0], Hydrothermal_801[0][1], color = "#f08575", ec="None", alpha =0.5)
    ax.axhspan(Hydrothermal_801[1][0], Hydrothermal_801[1][1], color = "#f08575", ec="None", alpha =0.5)
    # Breccia Unit
    ax.axhspan(Breccia_801[0], Breccia_801[1], color = "#84D2C5", ec="None", alpha =0.4)

    #### model reproducibility
    ### calculate
    model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
    #score_elem
    # 最小値を計算
    min_error_value = 1 * 10 ** (-model_reproducibility_elem_1)
    # 最大値を計算
    max_error_value = 1 * 10 ** model_reproducibility_elem_1
    ### calculate
    ax.axvspan(min_error_value, max_error_value, color = "#344c5c", ec="None", alpha =0.15)
    #### model reproducibility
    
    # Scatter plot settings
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.invert_yaxis()
    if elem_1=="P2O5":
        ax.set_xlabel("P mobility",  fontsize=30)
    else:
        ax.set_xlabel(elem_1+" mobility",  fontsize=30)
    ax.set_ylabel('Depth subbasement (m)', fontsize=30)
    ax.set_xscale('log')
    
    # KDE
    divider = make_axes_locatable(ax)
    ax_kde = divider.append_axes("top", size="20%", pad=0.0) 
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    
    sns.kdeplot(data=data.copy(), x=elem_1, ax=ax_kde, alpha=1, linewidth=3.5, log_scale=True, hue='Core_hole', hue_order=ALL_hue_order, palette=ALL_color, legend=False, common_norm=False)
    ax_kde.set(xlim=ax.get_xlim())
    ax_kde.set(xticks=[], yticks=[], xlabel=None, ylabel=None)
    ax_kde.xaxis.set_visible(False)

# 空白のサブプロットを非表示にする
if len(focus_element) < axes.size:
    for i in range(len(focus_element), axes.size):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.savefig('../Figure/4_depth/2_West_Focus.pdf', bbox_inches='tight')
plt.show()


## Scatter plot

In [ ]:
data=SA_mobility.loc[South_index].copy()
data_all = SA_mobility.copy()

for elem_1 in elem_list_now: # X axis
    for elem_2 in elem_list_now: # Y axis
        if not elem_1==elem_2:
            corr=data[[elem_1, elem_2]].apply(lambda x:np.log10(x)).corr().loc[elem_1][elem_2]

            plt.figure(figsize=(5, 5))
            plt.vlines(x=1, ymin=data_all[elem_2].min(), ymax=data_all[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
            plt.hlines(y=1, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=3.5)
            
            #### model reproducibility
            ### calculate
            model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
            model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
            #score_elem
            # 最大値・最小値を計算
            min_error_value_elem_1 = 1 * 10 ** (-model_reproducibility_elem_1)
            max_error_value_elem_1 = 1 * 10 ** model_reproducibility_elem_1
            min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
            max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
            ### calculate
            plt.axvspan(min_error_value_elem_1, max_error_value_elem_1, color = "#344c5c", ec="None", alpha =0.1) # X axis
            plt.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
            #### model reproducibility
            
            # データの準備
            data_main = data.copy().drop(list(featured_sample_color_map_S))
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]            
                plt.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

            ###### Featured samples
            for sample, color in featured_sample_color_map_S.items():
                try:
                    plt.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                               edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
                except:
                    pass
            ###### Featured samples
            ############################### Main 図
            
            plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True, labelsize=20)
            plt.tick_params(which = 'major', length = 7.5, width = 2)
            plt.tick_params(which = 'minor', length = 4, width = 1)
            plt.xscale('log')
            plt.yscale('log')
            
            if elem_1=="P2O5":
                ax.set_xlabel("P" +" mobility",  fontsize=30)
            else:
                ax.set_xlabel(elem_1+" mobility",  fontsize=30)
            if elem_2=="P2O5":
                ax.set_ylabel("P"+" mobility",  fontsize=30)
            else:
                ax.set_ylabel(elem_2+" mobility",  fontsize=30)

            ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)

            plt.savefig('../Figure/5_scatter/1_South_'+elem_1+ '_' +elem_2+ '.pdf', bbox_inches='tight')
            plt.show()

In [ ]:
data = SA_mobility.loc[South_index].copy()
data_all = SA_mobility.copy()

# 指定されたelem_1とelem_2の組み合わせ
selected_combinations = [('K', 'Rb'), ('K', 'Ba'), ('K', 'U'), ('K', 'Pb'), ('K', 'Sr'), ('K', 'P2O5'), ('Pb', 'P2O5')]

# 列数の計算
num_cols = min(len(selected_combinations), 4)

# 行数と列数に基づいて図のサイズを計算
fig_width = 5 * num_cols
fig_height = 5 * ((len(selected_combinations) - 1) // num_cols + 1)

fig, axes = plt.subplots(nrows=(len(selected_combinations) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

for i, (elem_1, elem_2) in enumerate(selected_combinations):
    row = i // num_cols
    col = i % num_cols
    
    corr = data[[elem_1, elem_2]].apply(lambda x: np.log10(x)).corr().loc[elem_1][elem_2]

    ax = axes[row, col]

    ax.vlines(x=1, ymin=data_all[elem_2].min(), ymax=data_all[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
    ax.hlines(y=1, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=3.5)

    #### model reproducibility
    ### calculate
    model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
    model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
    #score_elem
    # 最大値・最小値を計算
    min_error_value_elem_1 = 1 * 10 ** (-model_reproducibility_elem_1)
    max_error_value_elem_1 = 1 * 10 ** model_reproducibility_elem_1
    min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
    max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
    ### calculate
    ax.axvspan(min_error_value_elem_1, max_error_value_elem_1, color = "#344c5c", ec="None", alpha =0.1) # X axis
    ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
    #### model reproducibility
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_S))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_S.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図    
    
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.set_xscale('log')
    ax.set_yscale('log')

    if elem_1 == "P2O5":
        ax.set_xlabel("P"+" mobility", fontsize=30)
    else:
        ax.set_xlabel(elem_1+" mobility", fontsize=30)
    if elem_2 == "P2O5":
        ax.set_ylabel("P"+" mobility", fontsize=30)
    else:
        ax.set_ylabel(elem_2+" mobility", fontsize=30)

    ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)

# 空白のサブプロットを非表示にする
if len(selected_combinations) < axes.size:
    for i in range(len(selected_combinations), axes.size):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.savefig('../Figure/5_scatter/0_combination/1_South_selected_combinations.pdf', bbox_inches='tight')
plt.show()

In [ ]:
data=SA_mobility.loc[West_index].copy()
data_all = SA_mobility.copy()
for elem_1 in elem_list_now:
    for elem_2 in elem_list_now:
        if not elem_1==elem_2:
            corr=data[[elem_1, elem_2]].apply(lambda x:np.log10(x)).corr().loc[elem_1][elem_2]

            plt.figure(figsize=(5, 5))
            plt.vlines(x=1, ymin=data_all[elem_2].min(), ymax=data_all[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
            plt.hlines(y=1, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=3.5)
            
            # データの準備
            data_main = data.copy().drop(list(featured_sample_color_map_N))
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]            
                plt.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

            ###### Featured samples
            for sample, color in featured_sample_color_map_N.items():
                try:
                    plt.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                               edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
                except:
                    pass
            ###### Featured samples
            ############################### Main 図          
            #### model reproducibility
            ### calculate
            model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
            model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
            #score_elem
            # 最大値・最小値を計算
            min_error_value_elem_1 = 1 * 10 ** (-model_reproducibility_elem_1)
            max_error_value_elem_1 = 1 * 10 ** model_reproducibility_elem_1
            min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
            max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
            ### calculate
            ax.axvspan(min_error_value_elem_1, max_error_value_elem_1, color = "#344c5c", ec="None", alpha =0.1) # X axis
            ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
            #### model reproducibility
            
            plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True, labelsize=20)
            plt.tick_params(which = 'major', length = 7.5, width = 2)
            plt.tick_params(which = 'minor', length = 4, width = 1)
            plt.xscale('log')
            plt.yscale('log')
            
            if elem_1=="P2O5":
                ax.set_xlabel("P"+" mobility",  fontsize=30)
            else:
                ax.set_xlabel(elem_1+" mobility",  fontsize=30)
            if elem_2=="P2O5":
                ax.set_ylabel("P"+" mobility",  fontsize=30)
            else:
                ax.set_ylabel(elem_2+" mobility",  fontsize=30)  
            
            ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)
        plt.savefig('../Figure/5_scatter/2_West_'+elem_1+ '_' +elem_2+ '.pdf', bbox_inches='tight')
        plt.show()

In [ ]:
data = SA_mobility.loc[West_index].copy()
data_all = SA_mobility.copy()

# 指定されたelem_1とelem_2の組み合わせ
selected_combinations = [('K', 'Rb'), ('K', 'Ba'), ('K', 'U'), ('K', 'Pb'), ('K', 'Sr'), ('K', 'P2O5'), ('Pb', 'P2O5')]

# 列数の計算
num_cols = min(len(selected_combinations), 4)

# 行数と列数に基づいて図のサイズを計算
fig_width = 5 * num_cols
fig_height = 5 * ((len(selected_combinations) - 1) // num_cols + 1)

fig, axes = plt.subplots(nrows=(len(selected_combinations) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

for i, (elem_1, elem_2) in enumerate(selected_combinations):
    row = i // num_cols
    col = i % num_cols
    
    corr = data[[elem_1, elem_2]].apply(lambda x: np.log10(x)).corr().loc[elem_1][elem_2]

    ax = axes[row, col]

    ax.vlines(x=1, ymin=data_all[elem_2].min(), ymax=data_all[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
    ax.hlines(y=1, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=3.5)
    
    #### model reproducibility
    ### calculate
    model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
    model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
    #score_elem
    # 最大値・最小値を計算
    min_error_value_elem_1 = 1 * 10 ** (-model_reproducibility_elem_1)
    max_error_value_elem_1 = 1 * 10 ** model_reproducibility_elem_1
    min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
    max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
    ### calculate
    ax.axvspan(min_error_value_elem_1, max_error_value_elem_1, color = "#344c5c", ec="None", alpha =0.1) # X axis
    ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
    #### model reproducibility
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map_N))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map_N.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図       
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.set_xscale('log')
    ax.set_yscale('log')

    if elem_1 == "P2O5":
        ax.set_xlabel("P"+" mobility", fontsize=30)
    else:
        ax.set_xlabel(elem_1+" mobility", fontsize=30)
    if elem_2 == "P2O5":
        ax.set_ylabel("P"+" mobility", fontsize=30)
    else:
        ax.set_ylabel(elem_2+" mobility", fontsize=30)

    ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)

# 空白のサブプロットを非表示にする
if len(selected_combinations) < axes.size:
    for i in range(len(selected_combinations), axes.size):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.savefig('../Figure/5_scatter/0_combination/2_West_selected_combinations.pdf', bbox_inches='tight')
plt.show()

In [ ]:
data=SA_mobility.copy()
data_all = SA_mobility.copy()
for elem_1 in elem_list_now:
    for elem_2 in elem_list_now:
        if not elem_1==elem_2:
            corr=data[[elem_1, elem_2]].apply(lambda x:np.log10(x)).corr().loc[elem_1][elem_2]

            plt.figure(figsize=(5, 5))
            plt.vlines(x=1, ymin=data_all[elem_2].min(), ymax=data_all[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
            plt.hlines(y=1, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=3.5)
            
            # データの準備
            data_main = data.copy().drop(list(featured_sample_color_map))
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]            
                plt.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

            ###### Featured samples
            for sample, color in featured_sample_color_map.items():
                try:
                    plt.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                               edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
                except:
                    pass
            ###### Featured samples
            ############################### Main 図              
            #### model reproducibility
            ### calculate
            model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
            model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
            #score_elem
            # 最大値・最小値を計算
            min_error_value_elem_1 = 1 * 10 ** (-model_reproducibility_elem_1)
            max_error_value_elem_1 = 1 * 10 ** model_reproducibility_elem_1
            min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
            max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
            ### calculate
            ax.axvspan(min_error_value_elem_1, max_error_value_elem_1, color = "#344c5c", ec="None", alpha =0.1) # X axis
            ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
            #### model reproducibility
            
            plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True, labelsize=20)
            plt.tick_params(which = 'major', length = 7.5, width = 2)
            plt.tick_params(which = 'minor', length = 4, width = 1)
            plt.xscale('log')
            plt.yscale('log')
            
            if elem_1=="P2O5":
                ax.set_xlabel("P"+" mobility",  fontsize=30)
            else:
                ax.set_xlabel(elem_1+" mobility",  fontsize=30)
            if elem_2=="P2O5":
                ax.set_ylabel("P"+" mobility",  fontsize=30)
            else:
                ax.set_ylabel(elem_2+" mobility",  fontsize=30)  
            
            ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)
        plt.savefig('../Figure/5_scatter/3_ALL_'+elem_1+ '_' +elem_2+ '.pdf', bbox_inches='tight')
        plt.show()

In [ ]:
data = SA_mobility.copy()
data_all = SA_mobility.copy()

# 指定されたelem_1とelem_2の組み合わせ
selected_combinations = [('K', 'Rb'), ('K', 'Ba'), ('K', 'U'), ('K', 'Pb'), ('K', 'Sr'), ('K', 'P2O5'), ('Pb', 'P2O5')]

# 列数の計算
num_cols = min(len(selected_combinations), 4)

# 行数と列数に基づいて図のサイズを計算
fig_width = 5 * num_cols
fig_height = 5 * ((len(selected_combinations) - 1) // num_cols + 1)

fig, axes = plt.subplots(nrows=(len(selected_combinations) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

for i, (elem_1, elem_2) in enumerate(selected_combinations):
    row = i // num_cols
    col = i % num_cols
    
    corr = data[[elem_1, elem_2]].apply(lambda x: np.log10(x)).corr().loc[elem_1][elem_2]

    ax = axes[row, col]

    ax.vlines(x=1, ymin=data_all[elem_2].min(), ymax=data_all[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
    ax.hlines(y=1, xmin=data_all[elem_1].min(), xmax=data_all[elem_1].max(), linestyles=':', color='black', linewidth=3.5)
    
    #### model reproducibility
    ### calculate
    model_reproducibility_elem_1 = model_score.loc["Default_Test_mean"][elem_1]
    model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
    #score_elem
    # 最大値・最小値を計算
    min_error_value_elem_1 = 1 * 10 ** (-model_reproducibility_elem_1)
    max_error_value_elem_1 = 1 * 10 ** model_reproducibility_elem_1
    min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
    max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
    ### calculate
    ax.axvspan(min_error_value_elem_1, max_error_value_elem_1, color = "#344c5c", ec="None", alpha =0.1) # X axis
    ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
    #### model reproducibility
    
    # データの準備
    data_main = data.copy().drop(list(featured_sample_color_map))
    # データの準備
    ############################### Main 図
    # 各カテゴリに対して散布図を作成
    for category, category_c in zip(ALL_hue_order, ALL_color):
        category_data = data_main[data_main['Core_hole'] == category]            
        ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

    ###### Featured samples
    for sample, color in featured_sample_color_map.items():
        try:
            ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                       edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
        except:
            pass
    ###### Featured samples
    ############################### Main 図          
    ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
    ax.tick_params(which='major', length=7.5, width=2)
    ax.tick_params(which='minor', length=4, width=1)
    ax.set_xscale('log')
    ax.set_yscale('log')

    if elem_1 == "P2O5":
        ax.set_xlabel("P"+" mobility", fontsize=30)
    else:
        ax.set_xlabel(elem_1+" mobility", fontsize=30)
    if elem_2 == "P2O5":
        ax.set_ylabel("P"+" mobility", fontsize=30)
    else:
        ax.set_ylabel(elem_2+" mobility", fontsize=30)

    ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)

# 空白のサブプロットを非表示にする
if len(selected_combinations) < axes.size:
    for i in range(len(selected_combinations), axes.size):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.savefig('../Figure/5_scatter/0_combination/3_ALL_selected_combinations.pdf', bbox_inches='tight')
plt.show()

## Secondary mineral


In [ ]:
SA_Secoundary.columns

In [ ]:
data=SA_mobility.loc[South_index].copy()
data[SA_Secoundary.columns]= SA_Secoundary

Secoundary_list = SA_Secoundary.columns
float_check = pd.DataFrame(SA_Secoundary.dtypes=='float64')
float_col = float_check[float_check[0]==True].index.values

# Log flag 1=log, 0=normal
Log_flag = 0

for elem_2 in elem_list_now:
    for elem_1 in float_col:
        if not elem_1==elem_2:
            if Log_flag == 1:
                # log
                corr=data[[elem_1, elem_2]].apply(lambda x: np.log10(x)).corr().loc[elem_1][elem_2]
            else:
                # normal
                corr=data[[elem_1, elem_2]].corr().loc[elem_1][elem_2]

            plt.figure(figsize=(5, 5))
            #plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
            plt.hlines(y=1, xmin=data[elem_1].min(), xmax=data[elem_1].max(), linestyles=':', color='black', linewidth=3.5)
            
            # データの準備
            data_main = data.copy().drop(list(featured_sample_color_map_S))
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]            
                plt.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

            ###### Featured samples
            for sample, color in featured_sample_color_map_S.items():
                try:
                    plt.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                               edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
                except:
                    pass
            ###### Featured samples
            ############################### Main 図    
    
            #### model reproducibility
            ### calculate
            model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
            #score_elem
            # 最大値・最小値を計算
            min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
            max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
            ### calculate
            ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
            #### model reproducibility
            
            plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True, labelsize=20)
            plt.tick_params(which = 'major', length = 7.5, width = 2)
            plt.tick_params(which = 'minor', length = 4, width = 1)
            if Log_flag == 1:
                ax.set_xscale('log')
            plt.yscale('log')

            
            if elem_1=="P2O5":
                ax.set_xlabel("P",  fontsize=30)
            else:
                ax.set_xlabel(elem_1,  fontsize=30)
            if elem_2=="P2O5":
                ax.set_ylabel("P"+" mobility",  fontsize=30)
            else:
                ax.set_ylabel(elem_2+" mobility",  fontsize=30)

            ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)
        plt.savefig('../Figure/6_secondary_mineral/1_South_'+elem_1+ '_' +elem_2+ '.pdf', bbox_inches='tight')
        plt.show()

In [ ]:
data = SA_mobility.loc[South_index].copy()
data[SA_Secoundary.columns]= SA_Secoundary

focused_element = ['Rb', 'Ba', 'K', 'U', 'Pb', 'Sr', 'P2O5']
#focused_element = elem_list_now

Secoundary_list = SA_Secoundary.columns
float_check = pd.DataFrame(SA_Secoundary.dtypes=='float64')
float_col = float_check[float_check[0]==True].index.values

# Log flag 1=log, 0=normal
for Log_flag in [0, 1]:
    
    for elem_1 in float_col:
        # 列数の計算
        num_cols = min(len(focused_element), 4)

        # 行数と列数に基づいて図のサイズを計算
        fig_width = 6 * num_cols
        fig_height = 5.5 * ((len(focused_element) - 1) // num_cols + 1)

        fig, axes = plt.subplots(nrows=(len(focused_element) - 1) // num_cols + 1, ncols=num_cols, figsize=(fig_width, fig_height))

        for i, elem_2 in enumerate(focused_element):
            row = i // num_cols
            col = i % num_cols

            if Log_flag == 1:
                # log
                corr=data[[elem_1, elem_2]].apply(lambda x: np.log10(x)).corr().loc[elem_1][elem_2]
            else:
                # normal
                corr=data[[elem_1, elem_2]].corr().loc[elem_1][elem_2]    

            ax = axes[row, col]

            # plt.vlines(x=1, ymin=data[elem_2].min(), ymax=data[elem_2].max(), linestyles=':', color='black', linewidth=3.5)
            ax.hlines(y=1, xmin=data[elem_1].min(), xmax=data[elem_1].max(), linestyles=':', color='black', linewidth=3.5)
            
            #### model reproducibility
            ### calculate
            model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
            #score_elem
            # 最大値・最小値を計算
            min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
            max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
            ### calculate
            ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
            #### model reproducibility
            
            # データの準備
            data_main = data.copy().drop(list(featured_sample_color_map_S))
            # データの準備
            ############################### Main 図
            # 各カテゴリに対して散布図を作成
            for category, category_c in zip(ALL_hue_order, ALL_color):
                category_data = data_main[data_main['Core_hole'] == category]            
                ax.scatter(category_data[elem_1], category_data[elem_2], s=200, edgecolors=category_c, facecolors='none', marker="o", linewidth=5, zorder=10)

            ###### Featured samples
            for sample, color in featured_sample_color_map_S.items():
                try:
                    ax.scatter(data.loc[sample][elem_1], data.loc[sample][elem_2], \
                               edgecolors=color, facecolors='none', marker="d", linewidth=5, s=300, zorder=10)
                except:
                    pass
            ###### Featured samples
            ############################### Main 図                
            
            ax.tick_params(which='both', direction='in', bottom=True, left=True, top=True, right=True, labelsize=20)
            ax.tick_params(which='major', length=7.5, width=2)
            ax.tick_params(which='minor', length=4, width=1)
            if Log_flag == 1:
                ax.set_xscale('log')
            ax.set_yscale('log')

            if elem_1 == "P2O5":
                ax.set_xlabel("P", fontsize=25)
            else:
                ax.set_xlabel(elem_1, fontsize=25)
            if elem_2 == "P2O5":
                ax.set_ylabel("P"+" mobility", fontsize=25)
            else:
                ax.set_ylabel(elem_2+" mobility", fontsize=25)

            ax.text(0.95, 0.05, 'R={:.3f}'.format(corr), horizontalalignment='right', transform=ax.transAxes, fontsize = 20)

        # 空白のサブプロットを非表示にする
        if len(focused_element) < axes.size:
            for i in range(len(focused_element), axes.size):
                fig.delaxes(axes.flatten()[i])

        plt.tight_layout()
        plt.savefig('../Figure/6_secondary_mineral/0_combination/1_South_total_secondary_mineral_scatter_plots_'+ elem_1+str(Log_flag)+'.pdf', bbox_inches='tight')
        plt.show()

## Appendix

In [ ]:
    #### model reproducibility
    ### calculate
    model_reproducibility_elem_2 = model_score.loc["Default_Test_mean"][elem_2]
    #score_elem
    # 最大値・最小値を計算
    min_error_value_elem_2 = 1 * 10 ** (-model_reproducibility_elem_2)
    max_error_value_elem_2 = 1 * 10 ** model_reproducibility_elem_2
    ### calculate
    ax.axhspan(min_error_value_elem_2, max_error_value_elem_2, color = "#344c5c", ec="None", alpha =0.1) # Y axis
    #### model reproducibility

In [ ]:
# SA_mobilityデータを使用してheatmapを作成する場合
data = SA_mobility[elem_list_now].apply(lambda x: np.log10(x)).copy()

# heatmapを描画
plt.figure(figsize=(50, 50))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of SA_mobility', fontsize=16)
plt.xlabel('Element', fontsize=14)
plt.ylabel('Element', fontsize=14)
plt.show()

In [ ]:
SA_mobility